# Transformación de Unidades de Proyecto

Este notebook permite realizar el análisis y transformación de datos de **unidades de proyecto** de forma interactiva.

**Características:**
- Procesamiento de referencias (listas/strings)
- Limpieza de valores monetarios
- Generación de identificadores únicos (UPID)
- Validación y limpieza de datos
- Enfoque de programación funcional

**Versión:** 3.0 (Sin procesamiento geoespacial)

## 1. Importar Librerías

Importamos todas las librerías necesarias para el procesamiento de datos.

In [1]:
import os
import sys
import pandas as pd
import json
import numpy as np
from typing import Optional, Dict, List, Any, Tuple, Union, Callable
from datetime import datetime
from functools import reduce, partial, wraps
from pathlib import Path

print("✓ Librerías importadas exitosamente")

✓ Librerías importadas exitosamente


## 2. Funciones Utilitarias de Programación Funcional

Definimos funciones auxiliares para programación funcional (compose, pipe, curry).

In [2]:
def compose(*functions: Callable) -> Callable:
    """Compose multiple functions into a single function."""
    return reduce(lambda f, g: lambda x: f(g(x)), functions, lambda x: x)

def pipe(value: Any, *functions: Callable) -> Any:
    """Apply a sequence of functions to a value (pipe operator)."""
    return reduce(lambda acc, func: func(acc), functions, value)

def curry(func: Callable) -> Callable:
    """Convert a function to a curried version for partial application."""
    @wraps(func)
    def curried(*args, **kwargs):
        if len(args) + len(kwargs) >= func.__code__.co_argcount:
            return func(*args, **kwargs)
        return lambda *more_args, **more_kwargs: curried(*(args + more_args), **dict(kwargs, **more_kwargs))
    return curried

def safe_transform(func: Callable, fallback_value: Any = None) -> Callable:
    """Safely execute transformation functions with error handling."""
    @wraps(func)
    def wrapper(*args, **kwargs):
        try:
            return func(*args, **kwargs)
        except Exception as e:
            print(f"Warning in {func.__name__}: {e}")
            return fallback_value
    return wrapper

print("✓ Funciones utilitarias definidas")

✓ Funciones utilitarias definidas


## 3. Funciones de Limpieza de Datos

Funciones para limpieza de columnas numéricas, monetarias y de texto.

In [3]:
def clean_monetary_value(value):
    """Clean monetary values by removing currency symbols and thousands separators."""
    if pd.isna(value) or value is None:
        return 0.00
    
    if isinstance(value, (int, float)):
        numeric_value = float(value)
        if numeric_value < 0:
            print(f"    Warning: Negative monetary value {numeric_value} converted to 0.00")
            return 0.00
        return round(numeric_value, 2)
    
    str_value = str(value).strip()
    
    if str_value in ['-', '', 'nan', 'null', 'NaN', 'NULL', 'None']:
        return 0.00
    
    cleaned = str_value.replace('$', '').replace('COP', '').replace('USD', '').replace(' ', '').replace('\t', '').strip()
    
    if cleaned in ['-', '', '+']:
        return 0.00
    
    is_negative = cleaned.startswith('-')
    if is_negative:
        cleaned = cleaned[1:]
        print(f"    Warning: Negative monetary value '{str_value}' converted to positive")
    
    try:
        if '.' in cleaned and ',' not in cleaned:
            parts = cleaned.split('.')
            if len(parts) > 2 or (len(parts) == 2 and len(parts[1]) > 2):
                cleaned = cleaned.replace('.', '')
        elif ',' in cleaned and '.' in cleaned:
            cleaned = cleaned.replace(',', '')
        elif ',' in cleaned and '.' not in cleaned:
            comma_pos = cleaned.rfind(',')
            if len(cleaned) - comma_pos - 1 <= 2:
                cleaned = cleaned.replace(',', '.')
            else:
                cleaned = cleaned.replace(',', '')
        
        if not cleaned or cleaned == '.':
            return 0.00
        
        result = float(cleaned)
        
        if result < 0:
            print(f"    Warning: Negative result {result} from '{str_value}' converted to 0.00")
            result = 0.00
        
        return round(result, 2)
        
    except (ValueError, TypeError) as e:
        print(f"    Error cleaning monetary value '{str_value}': {e} - Setting to 0.00")
        return 0.00

def clean_numeric_column(df: pd.DataFrame, column_name: str, default_value: float = 0.0) -> pd.DataFrame:
    """Clean a numeric column using functional approach."""
    if column_name in df.columns:
        df = df.copy()
        df[column_name] = pd.to_numeric(df[column_name], errors='coerce').fillna(default_value)
    return df

def clean_monetary_column(df: pd.DataFrame, column_name: str, as_integer: bool = False) -> pd.DataFrame:
    """Clean a monetary column using functional approach."""
    if column_name in df.columns:
        df = df.copy()
        
        print(f"Cleaning monetary column: {column_name}")
        df[column_name] = df[column_name].apply(clean_monetary_value)
        df[column_name] = pd.to_numeric(df[column_name], errors='coerce').fillna(0.0)
        
        negative_mask = df[column_name] < 0
        if negative_mask.any():
            negative_count = negative_mask.sum()
            print(f"  Warning: Found {negative_count} negative values in {column_name}, converting to 0.00")
            df.loc[negative_mask, column_name] = 0.0
        
        positive_values = (df[column_name] > 0).sum()
        zero_values = (df[column_name] == 0).sum()
        total_values = len(df[column_name])
        
        print(f"  {column_name} validation results:")
        print(f"    Positive values: {positive_values}")
        print(f"    Zero values: {zero_values}")
        print(f"    Total values: {total_values}")
        
        if as_integer:
            df[column_name] = df[column_name].astype('int64')
        
    return df

print("✓ Funciones de limpieza definidas")

✓ Funciones de limpieza definidas


## 4. Funciones de Procesamiento de Referencias

Funciones para normalizar campos de referencia que pueden ser strings o listas.

In [4]:
def normalize_reference_value(value: Any) -> Optional[Union[str, List[str]]]:
    """Normalize reference values that can be either strings or lists."""
    if value is None:
        return None
    
    try:
        if not isinstance(value, (list, tuple, np.ndarray)) and pd.isna(value):
            return None
    except (ValueError, TypeError):
        pass
    
    try:
        if isinstance(value, str):
            value = value.strip()
            if value == '' or value.lower() in ['nan', 'null', 'none']:
                return None
            
            if value.startswith('[') and value.endswith(']'):
                try:
                    parsed = json.loads(value)
                    if isinstance(parsed, list):
                        filtered = [str(item).strip() for item in parsed if item and str(item).strip()]
                        if len(filtered) > 1:
                            return filtered
                        elif len(filtered) == 1:
                            return filtered[0]
                        else:
                            return None
                except json.JSONDecodeError:
                    pass
            
            if ',' in value and not value.startswith('http'):
                items = [item.strip() for item in value.split(',') if item.strip()]
                if len(items) > 1:
                    return items
                elif len(items) == 1:
                    return items[0]
                else:
                    return None
            
            return value
        
        elif isinstance(value, (list, tuple)):
            filtered = [str(item).strip() for item in value if item and str(item).strip() and str(item).lower() not in ['nan', 'null', 'none']]
            if len(filtered) > 1:
                return filtered
            elif len(filtered) == 1:
                return filtered[0]
            else:
                return None
        
        else:
            str_value = str(value).strip()
            return str_value if str_value and str_value.lower() not in ['nan', 'null', 'none'] else None
            
    except Exception as e:
        print(f"Warning: Error normalizing reference value '{value}': {e}")
        return str(value) if value else None

def process_reference_columns(df: pd.DataFrame) -> pd.DataFrame:
    """Process reference columns that can contain lists or strings."""
    result_df = df.copy()
    
    reference_columns = ['referencia_proceso', 'referencia_contrato', 'url_proceso']
    
    for col in reference_columns:
        if col in result_df.columns:
            print(f"Processing reference column: {col}")
            
            result_df[col] = result_df[col].apply(normalize_reference_value)
            
            single_count = result_df[col].apply(lambda x: isinstance(x, str)).sum()
            list_count = result_df[col].apply(lambda x: isinstance(x, list)).sum()
            null_count = result_df[col].isna().sum()
            
            print(f"  - Single values: {single_count}")
            print(f"  - List values: {list_count}")
            print(f"  - Null values: {null_count}")
    
    return result_df

print("✓ Funciones de procesamiento de referencias definidas")

✓ Funciones de procesamiento de referencias definidas


## 5. Funciones de Generación de UPID

Funciones para generar identificadores únicos (UPID) para cada proyecto.

In [5]:
def generate_upid_for_records(df: pd.DataFrame) -> pd.DataFrame:
    """
    Generate unique upid (Unidades de Proyecto ID) with format UNP-# for records without upid.
    Preserves existing upid values and ensures no duplicates.
    
    Args:
        df: DataFrame with potential upid column
        
    Returns:
        DataFrame with upid column populated
    """
    result_df = df.copy()
    
    # Ensure upid column exists
    if 'upid' not in result_df.columns:
        result_df['upid'] = None
    
    # Find existing upid values to determine next consecutive number
    existing_upids = set()
    max_consecutive = 0
    
    for upid in result_df['upid'].dropna():
        if isinstance(upid, str) and upid.startswith('UNP-'):
            existing_upids.add(upid)
            # Extract number from UNP-# format
            try:
                number_part = upid.replace('UNP-', '')
                if number_part.isdigit():
                    max_consecutive = max(max_consecutive, int(number_part))
            except (ValueError, AttributeError):
                continue
    
    # Generate upid for records without one
    new_upids_count = 0
    next_consecutive = max_consecutive + 1
    
    for idx in result_df.index:
        current_upid = result_df.at[idx, 'upid']
        
        # Only assign upid if it's null, empty, or NaN
        if pd.isna(current_upid) or current_upid is None or str(current_upid).strip() == '':
            new_upid = f"UNP-{next_consecutive}"
            
            # Ensure uniqueness (very unlikely but safety check)
            while new_upid in existing_upids:
                next_consecutive += 1
                new_upid = f"UNP-{next_consecutive}"
            
            result_df.at[idx, 'upid'] = new_upid
            existing_upids.add(new_upid)
            next_consecutive += 1
            new_upids_count += 1
    
    print(f"✓ UPID Generation:")
    print(f"  - Existing upids preserved: {len(existing_upids) - new_upids_count}")
    print(f"  - New upids generated: {new_upids_count}")
    print(f"  - Total upids: {len(existing_upids)}")
    print(f"  - Next available number: UNP-{next_consecutive}")
    
    return result_df

print("✓ Función de generación de UPID definida")

✓ Función de generación de UPID definida


## 6. Funciones de Columnas Calculadas

Funciones para agregar columnas computadas al DataFrame.

In [6]:
def add_computed_columns(df: pd.DataFrame) -> pd.DataFrame:
    """Add computed columns for metadata."""
    result_df = df.copy()
    
    # Add computed columns without modifying original data
    new_columns = {
        'processed_timestamp': datetime.now().isoformat()
    }
    
    for col, default_value in new_columns.items():
        result_df[col] = default_value
    
    print(f"✓ Added computed columns: {list(new_columns.keys())}")
    return result_df

print("✓ Función de columnas calculadas definida")

✓ Función de columnas calculadas definida


## 7. Funciones de Limpieza de Tipos de Datos

Funciones para limpiar columnas específicas según su tipo de dato.

In [7]:
def clean_text_column(df: pd.DataFrame, column: str) -> pd.DataFrame:
    """Clean a single text column using functional approach."""
    if column in df.columns:
        result_df = df.copy()
        result_df[column] = result_df[column].apply(
            lambda x: None if pd.isna(x) else str(x).strip()
        )
        return result_df
    return df


def clean_numeric_column_safe(df: pd.DataFrame, column: str) -> pd.DataFrame:
    """Clean a single numeric column using functional approach."""
    if column in df.columns:
        result_df = df.copy()
        result_df[column] = pd.to_numeric(result_df[column], errors='coerce').fillna(0.0)
        return result_df
    return df


def clean_integer_column(df: pd.DataFrame, column: str) -> pd.DataFrame:
    """Clean a single integer column using functional approach - converts to integers without decimals."""
    if column in df.columns:
        result_df = df.copy()
        # First convert to numeric, then to integer (removing decimals)
        result_df[column] = pd.to_numeric(result_df[column], errors='coerce').fillna(0.0).astype(int)
        return result_df
    return df


def clean_bpin_column(df: pd.DataFrame) -> pd.DataFrame:
    """Clean BPIN column specifically - keeps as string for alphanumeric codes or converts to integer."""
    if 'bpin' in df.columns:
        result_df = df.copy()
        # Try to convert to numeric, but if it fails, keep as string (for alphanumeric BPIN codes)
        def process_bpin(value):
            if pd.isna(value) or value is None:
                return None
            str_value = str(value).strip()
            if str_value == '' or str_value.lower() in ['nan', 'null']:
                return None
            # Try to convert to integer if it's purely numeric
            if str_value.replace('.', '').replace(',', '').isdigit():
                try:
                    return int(float(str_value.replace(',', '.')))
                except (ValueError, TypeError):
                    return str_value
            else:
                # Keep as string for alphanumeric codes
                return str_value
        
        result_df['bpin'] = result_df['bpin'].apply(process_bpin)
        return result_df
    return df


def clean_boolean_column(df: pd.DataFrame, column: str) -> pd.DataFrame:
    """Clean a single boolean column using functional approach."""
    if column in df.columns:
        result_df = df.copy()
        result_df[column] = result_df[column].astype(bool)
        return result_df
    return df

print("✓ Funciones de limpieza de tipos de datos definidas")

✓ Funciones de limpieza de tipos de datos definidas


## 8. Pipeline de Limpieza de Datos

Función principal que coordina todas las operaciones de limpieza usando composición funcional.

In [8]:
def clean_data_types(df: pd.DataFrame) -> pd.DataFrame:
    """Clean and standardize data types using functional composition."""
    
    # Define column types
    text_columns = ['nickname_detalle', 'direccion', 'descripcion_intervencion', 'identificador', 'nickname']
    # Variables monetarias que deben ser enteros (sin decimales)  
    integer_monetary_columns = ['presupuesto_base', 'ppto_base']
    # Variables numéricas enteras
    integer_columns = ['bpin']
    # Variables numéricas que pueden tener decimales
    decimal_columns = ['avance_obra', 'avance_fisico_obra']
    boolean_columns = ['centros_gravedad']
    
    # Create cleaning pipeline
    cleaning_functions = []
    
    # Add reference column processor first (handles complex list/string formats)
    cleaning_functions.append(process_reference_columns)
    
    # Add text column cleaners
    for col in text_columns:
        cleaning_functions.append(partial(clean_text_column, column=col))
    
    # Add monetary integer column cleaners (removes decimals from monetary values)
    for col in integer_monetary_columns:
        cleaning_functions.append(partial(clean_monetary_column, column_name=col, as_integer=True))
        
    # Add integer column cleaners (converts to integers)
    for col in integer_columns:
        cleaning_functions.append(partial(clean_integer_column, column=col))
    
    # Add decimal column cleaners (allows decimals)
    for col in decimal_columns:
        cleaning_functions.append(partial(clean_numeric_column_safe, column=col))
        
    # Add boolean column cleaners
    for col in boolean_columns:
        cleaning_functions.append(partial(clean_boolean_column, column=col))
    
    # Add BPIN column cleaner
    cleaning_functions.append(clean_bpin_column)
    
    # Apply all cleaning functions using functional composition
    return pipe(df, *cleaning_functions)

print("✓ Pipeline de limpieza de datos definido")

✓ Pipeline de limpieza de datos definido


## 9. Carga de Datos

In [9]:
import sys
from pathlib import Path

# Add extraction_app to path if not already there
# Define workspace path first
workspace_path = Path.cwd()  # or Path('/path/to/your/workspace')
extraction_app_path = workspace_path / 'extraction_app'

if str(extraction_app_path) not in sys.path:
    sys.path.insert(0, str(extraction_app_path))

# Import after adding the path
try:
    from data_extraction_unidades_proyecto import extract_unidades_proyecto_data as extract_unidades_proyecto
except ImportError:
    # If the function doesn't exist, try importing the module and check available functions
    import data_extraction_unidades_proyecto as data_module
    
    # List available functions in the module
    available_functions = [name for name in dir(data_module) if not name.startswith('_') and callable(getattr(data_module, name))]
    print(f"Available functions in data_extraction_unidades_proyecto: {available_functions}")
    
    # Try to find a function that extracts data
    if 'extract_data' in available_functions:
        extract_unidades_proyecto = data_module.extract_data
    elif 'load_data' in available_functions:
        extract_unidades_proyecto = data_module.load_data
    elif 'get_data' in available_functions:
        extract_unidades_proyecto = data_module.get_data
    else:
        raise ImportError(f"Cannot find a suitable extraction function. Available: {available_functions}")

print("=" * 60)
print("CARGANDO DATOS DE UNIDADES DE PROYECTO")
print("=" * 60)
print()

# Extract data using the extraction module
df = extract_unidades_proyecto()

print()
print("✓ Datos cargados exitosamente")
print(f"  - Total de registros: {len(df)}")
print(f"  - Total de columnas: {len(df.columns)}")
print()

🔧 Usando configuración de PRODUCCIÓN (.env.prod)
✅ Variables de entorno cargadas desde .env.prod
✅ Variables locales cargadas desde .env.local
CARGANDO DATOS DE UNIDADES DE PROYECTO

🚀 Extracting data from Google Drive Excel files directly to memory
FUNCTIONAL DATA EXTRACTION PIPELINE - GOOGLE DRIVE EXCEL FILES

1. Authenticating with Workload Identity Federation...
✅ Google Drive autenticado con Service Account
✓ Authentication successful with Workload Identity

2. Listing Excel files in Drive folder...
   Folder ID: 1q7CYewsqx***
✅ Encontrados 13 archivos Excel
   - Secretaría de Vivienda Social ...
   - Secretaría de Desarrollo Terri...
   - Secretaría de Educación.xlsx
   - Secretaría del Deporte y la Re...
   - Secretaría de Salud Pública.xl...
   - Secretaría de Cultura.xlsx
   - Secretaría de Bienestar Social...
   - Departamento Administrativo de...
   - Unidad Administrativa Especial...
   - Secretaría para la Gestión del...
   - Secretaría de Seguridad y Just...
   - Secretar

C:\Users\juanp\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


✅ Leído: Secretaría de Vivienda Social ... (939 filas, 28 columnas)

   [2/13] Processing: Secretaría de Desarrollo Territorial y Participación Ciudadana.xlsx
   Descargando Secretaría de Desarrollo Terri...: 100%
✅ Descargado: Secretaría de Desarrollo Terri...
✅ Leído: Secretaría de Desarrollo Terri... (10 filas, 28 columnas)

   [3/13] Processing: Secretaría de Educación.xlsx


C:\Users\juanp\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


   Descargando Secretaría de Educación.xlsx: 100%
✅ Descargado: Secretaría de Educación.xlsx
✅ Leído: Secretaría de Educación.xlsx (221 filas, 28 columnas)

   [4/13] Processing: Secretaría del Deporte y la Recreación.xlsx


...

C:\Users\juanp\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


   Descargando Secretaría de Paz y Cultura Ci...: 100%
✅ Descargado: Secretaría de Paz y Cultura Ci...
✅ Leído: Secretaría de Paz y Cultura Ci... (1 filas, 28 columnas)

   [13/13] Processing: Secretaría de Movilidad.xlsx


C:\Users\juanp\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


   Descargando Secretaría de Movilidad.xlsx: 100%
✅ Descargado: Secretaría de Movilidad.xlsx
✅ Leído: Secretaría de Movilidad.xlsx (4 filas, 28 columnas)

4. Concatenating data from all files...
✅ Concatenados 13 DataFrames:
   - Total de filas: 1641
   - Total de columnas: 28

5. Standardizing data structure...
✓ Data standardization complete: 29 columns

✓ Extraction completed successfully!
   - Files processed: 13
   - Total rows: 1641
   - Total columns: 29

✓ Extraction completed successfully!
  - Records extracted: 1641
  - Data ready for in-memory processing

✓ Datos cargados exitosamente
  - Total de registros: 1641
  - Total de columnas: 29



C:\Users\juanp\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [10]:
df.head()

,referencia_proceso,referencia_contrato,bpin,identificador,tipo_equipamiento,fuente_financiacion,nombre_up,nombre_up_detalle,comuna_corregimiento,tipo_intervencion,...,fecha_fin,lat,lon,plataforma,url_proceso,descripcion_intervencion,nombre_centro_gestor,microtio,dataframe,centros_gravedad
0,NaN,NaN,2024760010165,MISN Santa Elena,Vivienda nueva,Empréstito,MISN Santa Elena,Aportes al Macrorpoyecto de interes social nac...,COMUNA 18,Obra nueva,...,2025-10-30 00:00:00,3.441883,-76.520562,NaN,NaN,NaN,Secretaría de Vivienda Social y Habitat,NaN,NaN,False
1,4244.0.9.10.341-2025,NaN,2024760010165,Subsidios para cierre financiero,Vivienda nueva,Empréstito,Asignación del Subsidio Distrital de Vivienda ...,Asignación del Subsidio Distrital de Vivienda ...,EXPANSION URBANA,Obra nueva,...,2025-11-30 00:00:00,"3,327042","-76,503578",NaN,NaN,NaN,Secretaría de Vivienda Social y Habitat,NaN,NaN,False
2,4244.0.9.10.341-2025,NaN,2024760010165,Subsidios para cierre financiero,Vivienda nueva,Empréstito,Asignación del Subsidio Distrital de Vivienda ...,Asignación del Subsidio Distrital de Vivienda ...,COMUNA 16,Obra nueva,...,2025-11-30 00:00:00,"3,400405","-76,521555",NaN,NaN,NaN,Secretaría de Vivienda Social y Habitat,NaN,NaN,False
3,4244.0.9.10.341-2025,NaN,2024760010165,Subsidios para cierre financiero,Vivienda nueva,Empréstito,Asignación del Subsidio Distrital de Vivienda ...,Asignación del Subsidio Distrital de Vivienda ...,EXPANSION URBANA,Obra nueva,...,2025-11-30 00:00:00,"3,341943","-76,510784",NaN,NaN,NaN,Secretaría de Vivienda Social y Habitat,NaN,NaN,False
4,4244.0.9.10.341-2025,NaN,2024760010165,Subsidios para cierre financiero,Vivienda nueva,Empréstito,Asignación del Subsidio Distrital de Vivienda ...,Asignación del Subsidio Distrital de Vivienda ...,COMUNA 16,Obra nueva,...,2025-11-30 00:00:00,"3,400405","-76,521555",NaN,NaN,NaN,Secretaría de Vivienda Social y Habitat,NaN,NaN,False


## 10. Exploración Inicial de Datos

Analizar la estructura y contenido de los datos cargados.

In [11]:
# Display basic information
print("=" * 60)
print("INFORMACIÓN BÁSICA DEL DATASET")
print("=" * 60)
print(f"Número de registros: {len(df)}")
print(f"Número de columnas: {len(df.columns)}")
print()

# Show data types
print("Tipos de datos:")
print(df.dtypes)
print()

# Show shape
print(f"Shape: {df.shape}")
print()

# Display first few rows
print("Primeras 3 filas:")
df.head(3)

INFORMACIÓN BÁSICA DEL DATASET
Número de registros: 1641
Número de columnas: 29

Tipos de datos:
referencia_proceso           object
referencia_contrato          object
bpin                         object
identificador                object
tipo_equipamiento            object
fuente_financiacion          object
nombre_up                    object
nombre_up_detalle            object
comuna_corregimiento         object
tipo_intervencion            object
unidad                       object
cantidad                    float64
direccion                    object
barrio_vereda                object
estado                       object
presupuesto_base             object
avance_obra                  object
ano                         float64
fecha_inicio                 object
fecha_fin                    object
lat                          object
lon                          object
plataforma                   object
url_proceso                  object
descripcion_intervencion     object
nom

,referencia_proceso,referencia_contrato,bpin,identificador,tipo_equipamiento,fuente_financiacion,nombre_up,nombre_up_detalle,comuna_corregimiento,tipo_intervencion,...,fecha_fin,lat,lon,plataforma,url_proceso,descripcion_intervencion,nombre_centro_gestor,microtio,dataframe,centros_gravedad
0,NaN,NaN,2024760010165,MISN Santa Elena,Vivienda nueva,Empréstito,MISN Santa Elena,Aportes al Macrorpoyecto de interes social nac...,COMUNA 18,Obra nueva,...,2025-10-30 00:00:00,3.441883,-76.520562,NaN,NaN,NaN,Secretaría de Vivienda Social y Habitat,NaN,NaN,False
1,4244.0.9.10.341-2025,NaN,2024760010165,Subsidios para cierre financiero,Vivienda nueva,Empréstito,Asignación del Subsidio Distrital de Vivienda ...,Asignación del Subsidio Distrital de Vivienda ...,EXPANSION URBANA,Obra nueva,...,2025-11-30 00:00:00,"3,327042","-76,503578",NaN,NaN,NaN,Secretaría de Vivienda Social y Habitat,NaN,NaN,False
2,4244.0.9.10.341-2025,NaN,2024760010165,Subsidios para cierre financiero,Vivienda nueva,Empréstito,Asignación del Subsidio Distrital de Vivienda ...,Asignación del Subsidio Distrital de Vivienda ...,COMUNA 16,Obra nueva,...,2025-11-30 00:00:00,"3,400405","-76,521555",NaN,NaN,NaN,Secretaría de Vivienda Social y Habitat,NaN,NaN,False


In [12]:
# Check for missing values
print("=" * 60)
print("VALORES NULOS POR COLUMNA")
print("=" * 60)
null_counts = df.isnull().sum()
null_counts = null_counts[null_counts > 0].sort_values(ascending=False)
if len(null_counts) > 0:
    print(null_counts)
else:
    print("No hay valores nulos en el dataset")
print()

# Show some statistics for numeric columns
print("=" * 60)
print("ESTADÍSTICAS DE COLUMNAS NUMÉRICAS")
print("=" * 60)
df.describe()

VALORES NULOS POR COLUMNA
microtio                    1635
dataframe                   1578
descripcion_intervencion    1284
url_proceso                 1009
plataforma                   996
referencia_contrato          958
fecha_inicio                 259
unidad                       257
cantidad                     257
fecha_fin                    247
ano                          233
direccion                    178
lat                           78
lon                           78
bpin                          72
referencia_proceso            47
tipo_equipamiento             34
nombre_centro_gestor          33
identificador                 26
presupuesto_base              17
avance_obra                    5
nombre_up_detalle              5
barrio_vereda                  2
tipo_intervencion              1
dtype: int64

ESTADÍSTICAS DE COLUMNAS NUMÉRICAS


,cantidad,ano
count,1384.0,1408.000000
mean,1.0,2024.905540
std,0.0,0.463629
min,1.0,2024.000000
25%,1.0,2025.000000
50%,1.0,2025.000000
75%,1.0,2025.000000
max,1.0,2026.000000


In [13]:
df.drop(columns=['MicroTIO', 'dataframe', 'microtio', 'centros_gravedad'], inplace=True, errors='ignore')

## 11. Aplicar Pipeline de Transformación

Ejecutar todas las transformaciones sobre los datos usando composición funcional.

In [14]:
# Create transformation pipeline
print("=" * 60)
print("EJECUTANDO PIPELINE DE TRANSFORMACIÓN")
print("=" * 60)
print()

# Define the transformation pipeline using functional composition
transformation_pipeline = compose(
    generate_upid_for_records,
    add_computed_columns,
    clean_data_types
)

# Apply the pipeline
df_transformed = transformation_pipeline(df)

print()
print("=" * 60)
print("✓ TRANSFORMACIÓN COMPLETADA")
print("=" * 60)

EJECUTANDO PIPELINE DE TRANSFORMACIÓN

Processing reference column: referencia_proceso
  - Single values: 1521
  - List values: 73
  - Null values: 47
Processing reference column: referencia_contrato
  - Single values: 632
  - List values: 51
  - Null values: 958
Processing reference column: url_proceso
  - Single values: 632
  - List values: 0
  - Null values: 1009
Cleaning monetary column: presupuesto_base
  presupuesto_base validation results:
    Positive values: 1624
    Zero values: 17
    Total values: 1641
✓ Added computed columns: ['processed_timestamp']
✓ UPID Generation:
  - Existing upids preserved: 0
  - New upids generated: 1641
  - Total upids: 1641
  - Next available number: UNP-1642

✓ TRANSFORMACIÓN COMPLETADA


## 12. Validar Datos Transformados

Revisar la estructura y calidad de los datos después de la transformación.

In [15]:
# Display summary statistics
print("=" * 60)
print("RESUMEN DE DATOS TRANSFORMADOS")
print("=" * 60)
print(f"✓ Processed data: {len(df_transformed)} records")
print(f"✓ Total columns: {len(df_transformed.columns)}")
print()

# Count column types
text_cols = df_transformed.select_dtypes(include=['object']).columns
numeric_cols = df_transformed.select_dtypes(include=['number']).columns
bool_cols = df_transformed.select_dtypes(include=['bool']).columns

print(f"Column type distribution:")
print(f"  - Text columns: {len(text_cols)}")
print(f"  - Numeric columns: {len(numeric_cols)}")
print(f"  - Boolean columns: {len(bool_cols)}")
print()

# Show sample of transformed data
print("Sample de datos transformados (primeras 3 filas):")
df_transformed.head()

RESUMEN DE DATOS TRANSFORMADOS
✓ Processed data: 1641 records
✓ Total columns: 28

Column type distribution:
  - Text columns: 23
  - Numeric columns: 5
  - Boolean columns: 0

Sample de datos transformados (primeras 3 filas):


,referencia_proceso,referencia_contrato,bpin,identificador,tipo_equipamiento,fuente_financiacion,nombre_up,nombre_up_detalle,comuna_corregimiento,tipo_intervencion,...,fecha_inicio,fecha_fin,lat,lon,plataforma,url_proceso,descripcion_intervencion,nombre_centro_gestor,processed_timestamp,upid
0,None,None,2024760010165,MISN Santa Elena,Vivienda nueva,Empréstito,MISN Santa Elena,Aportes al Macrorpoyecto de interes social nac...,COMUNA 18,Obra nueva,...,03-08-2025,2025-10-30 00:00:00,3.441883,-76.520562,NaN,None,None,Secretaría de Vivienda Social y Habitat,2025-11-18T00:50:28.595762,UNP-1
1,4244.0.9.10.341-2025,None,2024760010165,Subsidios para cierre financiero,Vivienda nueva,Empréstito,Asignación del Subsidio Distrital de Vivienda ...,Asignación del Subsidio Distrital de Vivienda ...,EXPANSION URBANA,Obra nueva,...,24-06-2025,2025-11-30 00:00:00,"3,327042","-76,503578",NaN,None,None,Secretaría de Vivienda Social y Habitat,2025-11-18T00:50:28.595762,UNP-2
2,4244.0.9.10.341-2025,None,2024760010165,Subsidios para cierre financiero,Vivienda nueva,Empréstito,Asignación del Subsidio Distrital de Vivienda ...,Asignación del Subsidio Distrital de Vivienda ...,COMUNA 16,Obra nueva,...,24-06-2025,2025-11-30 00:00:00,"3,400405","-76,521555",NaN,None,None,Secretaría de Vivienda Social y Habitat,2025-11-18T00:50:28.595762,UNP-3
3,4244.0.9.10.341-2025,None,2024760010165,Subsidios para cierre financiero,Vivienda nueva,Empréstito,Asignación del Subsidio Distrital de Vivienda ...,Asignación del Subsidio Distrital de Vivienda ...,EXPANSION URBANA,Obra nueva,...,24-06-2025,2025-11-30 00:00:00,"3,341943","-76,510784",NaN,None,None,Secretaría de Vivienda Social y Habitat,2025-11-18T00:50:28.595762,UNP-4
4,4244.0.9.10.341-2025,None,2024760010165,Subsidios para cierre financiero,Vivienda nueva,Empréstito,Asignación del Subsidio Distrital de Vivienda ...,Asignación del Subsidio Distrital de Vivienda ...,COMUNA 16,Obra nueva,...,24-06-2025,2025-11-30 00:00:00,"3,400405","-76,521555",NaN,None,None,Secretaría de Vivienda Social y Habitat,2025-11-18T00:50:28.595762,UNP-5


In [16]:
# Check specific columns
print("=" * 60)
print("VERIFICACIÓN DE COLUMNAS ESPECÍFICAS")
print("=" * 60)
print()

# Check UPID column
if 'upid' in df_transformed.columns:
    print("✓ Columna UPID:")
    print(f"  - Total: {df_transformed['upid'].notna().sum()}")
    print(f"  - Sample: {df_transformed['upid'].head(5).tolist()}")
    print()

# Check processed_timestamp
if 'processed_timestamp' in df_transformed.columns:
    print("✓ Columna processed_timestamp:")
    print(f"  - Sample: {df_transformed['processed_timestamp'].iloc[0]}")
    print()

# Check monetary columns
monetary_cols = ['presupuesto_base', 'ppto_base']
for col in monetary_cols:
    if col in df_transformed.columns:
        print(f"✓ Columna {col}:")
        print(f"  - Tipo: {df_transformed[col].dtype}")
        print(f"  - Valores positivos: {(df_transformed[col] > 0).sum()}")
        print(f"  - Valores cero: {(df_transformed[col] == 0).sum()}")
        print(f"  - Total: ${df_transformed[col].sum():,.0f}")
        print()

VERIFICACIÓN DE COLUMNAS ESPECÍFICAS

✓ Columna UPID:
  - Total: 1641
  - Sample: ['UNP-1', 'UNP-2', 'UNP-3', 'UNP-4', 'UNP-5']

✓ Columna processed_timestamp:
  - Sample: 2025-11-18T00:50:28.595762

✓ Columna presupuesto_base:
  - Tipo: int64
  - Valores positivos: 1624
  - Valores cero: 17
  - Total: $700,978,737,519



In [17]:
df_transformed.columns

Index(['referencia_proceso', 'referencia_contrato', 'bpin', 'identificador',
       'tipo_equipamiento', 'fuente_financiacion', 'nombre_up',
       'nombre_up_detalle', 'comuna_corregimiento', 'tipo_intervencion',
       'unidad', 'cantidad', 'direccion', 'barrio_vereda', 'estado',
       'presupuesto_base', 'avance_obra', 'ano', 'fecha_inicio', 'fecha_fin',
       'lat', 'lon', 'plataforma', 'url_proceso', 'descripcion_intervencion',
       'nombre_centro_gestor', 'processed_timestamp', 'upid'],
      dtype='object')

In [18]:
df_transformed['estado'].unique()

array(['En alistamiento', 'En ejecución', 'En liquidación', 'Finalizado',
       'Socialización'], dtype=object)

In [19]:
# Replace "Socialización, En alistamiento" with "En alistamiento" in the estado column
df_transformed['estado'] = df_transformed['estado'].replace('Socialización', 'En alistamiento')

In [20]:
df_transformed['tipo_intervencion'].unique()

array(['Obra nueva', nan, 'Rehabilitación / Reforzamiento',
       'Adecuaciones y mantenimientos', 'Adecuaciones', 'Mantenimiento',
       'Rehabilitación - Reforzamiento'], dtype=object)

In [21]:
# Replace "Socialización, En alistamiento" with "En alistamiento" in the estado column
df_transformed['tipo_intervencion'] = df_transformed['tipo_intervencion'].replace('Adecuaciones', 'Adecuaciones y Mantenimientos')
df_transformed['tipo_intervencion'] = df_transformed['tipo_intervencion'].replace('Mantenimiento', 'Adecuaciones y Mantenimientos')
df_transformed['tipo_intervencion'] = df_transformed['tipo_intervencion'].replace('Rehabilitación / Reforzamiento', 'Rehabilitación - Reforzamiento')

## 13. Validar Coherencia Semántica de los datos

In [22]:
df_transformed['nombre_up'].unique()

array(['MISN Santa Elena',
       'Asignación del Subsidio Distrital de Vivienda para el cierre financiero ',
       'Socialización y difusión de la oferta habitacional ',
       'Estudios & Diseños Bulevar Figueroa',
       'Estudios & Diseños Cristo Rey etapa 4',
       'Bocatoma CAV Cristo Rey ', 'Comuna 13 Calipso',
       'Ciudadela Recreativa Pondaje y Charco Azul ',
       'Adquisición suelo Renovación Urbana  Mz 167 Plan Parcial San Pascual ',
       'Adquisición suelo Renovación Urbana /Usos Complementarios Mz 203 Plan Parcial Ciudadela La Justicia ',
       'Adquisición de Suelo Pizamos I  ',
       'Desarrollo predios SAE - Transferidos',
       'Construcción espacio publico manzana usos complementarios Fiscalía',
       'Compra de predios Cristo Rey',
       'Subsidios de Mejoramiento de Vivienda',
       'Estudios & Diseños Muros de Contención Comuna 1 & 20',
       'interventoria Parques comuna 18', 'Parques comuna 18',
       'Adecuación del Espacio Publico (Sentencia re

In [23]:
# Update the title_case_spanish function to handle UTS prefix correctly
def title_case_spanish(text: str) -> str:
    """
    Convert text to title case following Spanish conventions.
    - Capitalizes first letter of each word
    - Keeps connectors (de, y, para, etc.) in lowercase
    - Preserves known acronyms and institutional terms in uppercase
    - Special handling for UTS prefix (only UTS in uppercase, rest in title case)
    - Handles None and empty strings
    
    Args:
        text: String to convert to title case
        
    Returns:
        String with proper title case formatting
    """
    if pd.isna(text) or text is None or str(text).strip() == '':
        return text
    
    text = str(text).strip()
    
    # Spanish connectors and articles that should remain lowercase
    connectors = {
        'a', 'ante', 'bajo', 'con', 'contra', 'de', 'del', 'desde', 'durante',
        'e', 'el', 'en', 'entre', 'hacia', 'hasta', 'la', 'las', 'lo', 'los',
        'mediante', 'para', 'por', 'según', 'sin', 'sobre', 'tras', 'y', 'o', 'u', 'mi'
    }
    
    # Known acronyms and institutional terms (preserve uppercase)
    acronyms = {
        'ie', 'i.e', 'i.e.', 'ips', 'eps', 'uts', 'cad', 'secop', 'bpin', 'upid',
        'tic', 'tio', 'rrhh', 'pqrs', 'sst', 'covid', 'onu', 'oit', 'dian',
        'dane', 'dnp', 'sgr', 'poa', 'poai', 'iva', 'nit', 'rut', 'sisben'
    }
    
    words = text.split()
    result = []
    
    for i, word in enumerate(words):
        # Check if word is an acronym (case-insensitive)
        word_lower = word.lower().replace('.', '')
        
        if word_lower in acronyms:
            # Preserve original format or uppercase for acronyms
            result.append(word.upper())
        elif i > 0 and word_lower in connectors:
            # Keep connectors lowercase (except first word)
            result.append(word.lower())
        else:
            # Normal title case
            result.append(word.capitalize())
    
    return ' '.join(result)


# Apply title case to relevant columns in df_transformed
print("=" * 60)
print("APLICANDO TITLE CASE A COLUMNAS DE TEXTO")
print("=" * 60)
print()

columns_to_transform = ['nombre_up', 'nombre_up_detalle', 'direccion', 'tipo_equipamiento', 'identificador']

for col in columns_to_transform:
    if col in df_transformed.columns:
        print(f"Procesando columna: {col}")
        df_transformed[col] = df_transformed[col].apply(title_case_spanish)
        print(f"  ✓ {col} transformada")
        print(f"  Ejemplo: {df_transformed[col].iloc[0]}")
        print()
    else:
        print(f"⚠ Columna '{col}' no encontrada en el dataframe")
        print()

print("✓ Transformación de title case completada")


APLICANDO TITLE CASE A COLUMNAS DE TEXTO

Procesando columna: nombre_up
  ✓ nombre_up transformada
  Ejemplo: Misn Santa Elena

Procesando columna: nombre_up_detalle
  ✓ nombre_up_detalle transformada
  Ejemplo: Aportes Al Macrorpoyecto de Interes Social Nacional Santa Elenea a Través de la Asginación de Subsidios en Especie

Procesando columna: direccion
  ✓ direccion transformada
  Ejemplo: Calle 1a Oeste

Procesando columna: tipo_equipamiento
  ✓ tipo_equipamiento transformada
  Ejemplo: Vivienda Nueva

Procesando columna: identificador
  ✓ identificador transformada
  Ejemplo: Misn Santa Elena

✓ Transformación de title case completada


In [24]:
df_transformed

,referencia_proceso,referencia_contrato,bpin,identificador,tipo_equipamiento,fuente_financiacion,nombre_up,nombre_up_detalle,comuna_corregimiento,tipo_intervencion,...,fecha_inicio,fecha_fin,lat,lon,plataforma,url_proceso,descripcion_intervencion,nombre_centro_gestor,processed_timestamp,upid
0,None,None,2024760010165,Misn Santa Elena,Vivienda Nueva,Empréstito,Misn Santa Elena,Aportes Al Macrorpoyecto de Interes Social Nac...,COMUNA 18,Obra nueva,...,03-08-2025,2025-10-30 00:00:00,3.441883,-76.520562,NaN,None,None,Secretaría de Vivienda Social y Habitat,2025-11-18T00:50:28.595762,UNP-1
1,4244.0.9.10.341-2025,None,2024760010165,Subsidios para Cierre Financiero,Vivienda Nueva,Empréstito,Asignación del Subsidio Distrital de Vivienda ...,Asignación del Subsidio Distrital de Vivienda ...,EXPANSION URBANA,Obra nueva,...,24-06-2025,2025-11-30 00:00:00,"3,327042","-76,503578",NaN,None,None,Secretaría de Vivienda Social y Habitat,2025-11-18T00:50:28.595762,UNP-2
2,4244.0.9.10.341-2025,None,2024760010165,Subsidios para Cierre Financiero,Vivienda Nueva,Empréstito,Asignación del Subsidio Distrital de Vivienda ...,Asignación del Subsidio Distrital de Vivienda ...,COMUNA 16,Obra nueva,...,24-06-2025,2025-11-30 00:00:00,"3,400405","-76,521555",NaN,None,None,Secretaría de Vivienda Social y Habitat,2025-11-18T00:50:28.595762,UNP-3
3,4244.0.9.10.341-2025,None,2024760010165,Subsidios para Cierre Financiero,Vivienda Nueva,Empréstito,Asignación del Subsidio Distrital de Vivienda ...,Asignación del Subsidio Distrital de Vivienda ...,EXPANSION URBANA,Obra nueva,...,24-06-2025,2025-11-30 00:00:00,"3,341943","-76,510784",NaN,None,None,Secretaría de Vivienda Social y Habitat,2025-11-18T00:50:28.595762,UNP-4
4,4244.0.9.10.341-2025,None,2024760010165,Subsidios para Cierre Financiero,Vivienda Nueva,Empréstito,Asignación del Subsidio Distrital de Vivienda ...,Asignación del Subsidio Distrital de Vivienda ...,COMUNA 16,Obra nueva,...,24-06-2025,2025-11-30 00:00:00,"3,400405","-76,521555",NaN,None,None,Secretaría de Vivienda Social y Habitat,2025-11-18T00:50:28.595762,UNP-5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1636,4164.010.32.1.571-2024,4164.010.26.1.578-2024,2024760010015,Casa de las Memorias,Infraestructura Cultural,Recursos propios,Casa de las Memorias,Casa de las Memorias,COMUNA 03,Obra nueva,...,12-nov-24,2024-12-12 00:00:00,3.450352,-76.536379,SECOP II,https://community.secop.gov.co/Public/Tenderin...,None,Secretaría de Paz y Cultura Ciudadana,2025-11-18T00:50:28.595762,UNP-1637
1637,OC143697,OC143697,2024760010008,Demarcacion_plastico_frio,Señalización Vial,Ingresos con destinación específica,Demarcacion_plastico_frio,Adquirir Insumos para la Demarcacion de Vias d...,COMUNAS DE CALI,Obra nueva,...,2025/04/02,2025/07/20,NaN,NaN,Tienda Virtual,https://operaciones.colombiacompra.gov.co/tien...,None,Secretaría de Movilidad,2025-11-18T00:50:28.595762,UNP-1638
1638,OC149988,OC149988,2024760010107,Demarcacion_cicloinfraestrutura,Señalización Vial,Ingresos con destinación específica,Demarcacion_cicloinfraestrutura,Suministro de Insumos para la Demarcacion de C...,COMUNAS DE CALI,Obra nueva,...,2025/08/26,2025/10/31,NaN,NaN,Tienda Virtual,https://operaciones.colombiacompra.gov.co/tien...,None,Secretaría de Movilidad,2025-11-18T00:50:28.595762,UNP-1639
1639,4152.010.26.1.598-2025,4152.010.26.1.598-2025,2024760010008,Reductores,Señalización Vial,Ingresos con destinación específica,Reductores,"Construccion, Instalacion, Demolicion, y Repos...",COMUNAS DE CALI,Obra nueva,...,2025/07/14,2025/12/31,NaN,NaN,SECOP II,https://community.secop.gov.co/Public/Tenderin...,None,Secretaría de Movilidad,2025-11-18T00:50:28.595762,UNP-1640


In [25]:
#df_transformed['nombre_up'].unique()

## 14. Corrección de Datos Geográficos

### 14.1. Convertir df_transformed a un gdf (geodataframe)

In [26]:
import geopandas as gpd
from shapely.geometry import Point

# Create a copy to avoid modifying the original dataframe
gdf = df_transformed.copy()

# Clean and convert lat/lon columns to numeric
def safe_convert_to_float(value):
    """Safely convert string coordinates to float."""
    if pd.isna(value) or value is None:
        return None
    try:
        # Remove any whitespace and convert to float
        return float(str(value).strip())
    except (ValueError, TypeError):
        return None

# Apply conversion to lat/lon columns
if 'lat' in gdf.columns and 'lon' in gdf.columns:
    print("=" * 60)
    print("CONVIRTIENDO DATAFRAME A GEODATAFRAME")
    print("=" * 60)
    print()
    
    # Convert lat/lon to numeric
    gdf['lat_numeric'] = gdf['lat'].apply(safe_convert_to_float)
    gdf['lon_numeric'] = gdf['lon'].apply(safe_convert_to_float)
    
    # Filter rows with valid coordinates
    valid_coords = gdf['lat_numeric'].notna() & gdf['lon_numeric'].notna()
    
    print(f"Registros totales: {len(gdf)}")
    print(f"Registros con coordenadas válidas: {valid_coords.sum()}")
    print(f"Registros sin coordenadas: {(~valid_coords).sum()}")
    print()
    
    # Create geometry column for valid coordinates
    gdf.loc[valid_coords, 'geometry'] = gdf.loc[valid_coords].apply(
        lambda row: Point(row['lon_numeric'], row['lat_numeric']), 
        axis=1
    )
    
    # Convert to GeoDataFrame
    gdf = gpd.GeoDataFrame(gdf, geometry='geometry', crs='EPSG:4326')
    
    print(f"✓ GeoDataFrame creado exitosamente")
    print(f"✓ CRS: {gdf.crs}")
    print(f"✓ Geometrías válidas: {gdf['geometry'].notna().sum()}")
    print()
    
    # Display sample
    print("Muestra de geometrías:")
    print(gdf[gdf['geometry'].notna()][['upid', 'nombre_up', 'geometry']].head(3))
else:
    print("⚠ Warning: Las columnas 'lat' y 'lon' no se encontraron en el DataFrame")
    gdf = None

CONVIRTIENDO DATAFRAME A GEODATAFRAME

Registros totales: 1641
Registros con coordenadas válidas: 1066
Registros sin coordenadas: 575

✓ GeoDataFrame creado exitosamente
✓ CRS: EPSG:4326
✓ Geometrías válidas: 1066

Muestra de geometrías:
      upid                              nombre_up                   geometry
0    UNP-1                       Misn Santa Elena  POINT (-76.52056 3.44188)
60  UNP-61    Estudios & Diseños Bulevar Figueroa  POINT (-76.48431 3.41822)
61  UNP-62  Estudios & Diseños Cristo Rey Etapa 4  POINT (-76.56114 3.43553)


In [27]:
#gdf.head()

### 14.2. Corregir el formato de las coordenadas

Asegurando que los valores de "lat", siempre inicien por "3.xxx" y "lon" siempre comience por "-76.xxx".

In [28]:
def fix_coordinate_format(coord_value, coord_type='lat'):
    """
    Fix coordinate format ensuring proper structure.
    - lat should always start with "3."
    - lon should always start with "-76."
    
    Args:
        coord_value: Coordinate value to fix (can be string, float, or None)
        coord_type: Either 'lat' or 'lon'
        
    Returns:
        Properly formatted coordinate as float, or None if invalid
    """
    if pd.isna(coord_value) or coord_value is None:
        return None
    
    try:
        # Convert to string and clean
        coord_str = str(coord_value).strip()
        
        # Remove any whitespace
        coord_str = coord_str.replace(' ', '')
        
        # Handle comma as decimal separator
        if ',' in coord_str and '.' not in coord_str:
            coord_str = coord_str.replace(',', '.')
        
        # Convert to float
        coord_float = float(coord_str)
        
        if coord_type == 'lat':
            # Latitude should be between 3.0 and 4.0 for Cali region
            if coord_float >= 3.0 and coord_float <= 4.0:
                return round(coord_float, 10)
            # If it's missing the "3." prefix, add it
            elif coord_float > 0 and coord_float < 1:
                return round(3.0 + coord_float, 10)
            else:
                print(f"  Warning: Invalid latitude value {coord_float}, setting to None")
                return None
                
        elif coord_type == 'lon':
            # Longitude should be between -77.0 and -76.0 for Cali region
            if coord_float >= -77.0 and coord_float <= -76.0:
                return round(coord_float, 10)
            # If it's positive, make it negative
            elif coord_float > 76.0 and coord_float < 77.0:
                return round(-coord_float, 10)
            # If it's missing the "-76." prefix
            elif coord_float > 0 and coord_float < 1:
                return round(-76.0 - coord_float, 10)
            else:
                print(f"  Warning: Invalid longitude value {coord_float}, setting to None")
                return None
                
    except (ValueError, TypeError) as e:
        print(f"  Error converting coordinate '{coord_value}': {e}")
        return None


# Apply coordinate format correction to gdf
print("=" * 60)
print("CORRIGIENDO FORMATO DE COORDENADAS")
print("=" * 60)
print()

if gdf is not None and 'lat' in gdf.columns and 'lon' in gdf.columns:
    # Fix latitude values
    print("Corrigiendo valores de latitud...")
    gdf['lat_fixed'] = gdf['lat'].apply(lambda x: fix_coordinate_format(x, 'lat'))
    
    # Fix longitude values
    print("Corrigiendo valores de longitud...")
    gdf['lon_fixed'] = gdf['lon'].apply(lambda x: fix_coordinate_format(x, 'lon'))
    
    # Update the original lat/lon columns
    gdf['lat'] = gdf['lat_fixed']
    gdf['lon'] = gdf['lon_fixed']
    
    # Recreate geometry with fixed coordinates
    valid_coords_fixed = gdf['lat'].notna() & gdf['lon'].notna()
    
    print()
    print("Recreando geometrías con coordenadas corregidas...")
    gdf.loc[valid_coords_fixed, 'geometry'] = gdf.loc[valid_coords_fixed].apply(
        lambda row: Point(row['lon'], row['lat']), 
        axis=1
    )
    
    # Update geodataframe
    gdf = gpd.GeoDataFrame(gdf, geometry='geometry', crs='EPSG:4326')
    
    print()
    print("✓ CORRECCIÓN COMPLETADA")
    print("=" * 60)
    print(f"Registros totales: {len(gdf)}")
    print(f"Coordenadas válidas después de corrección: {valid_coords_fixed.sum()}")
    print(f"Coordenadas inválidas: {(~valid_coords_fixed).sum()}")
    print()
    
    # Show sample of corrected coordinates
    print("Muestra de coordenadas corregidas:")
    sample_df = gdf[valid_coords_fixed][['upid', 'nombre_up', 'lat', 'lon']].head(5)
    print(sample_df)
    
    # Clean up temporary columns
    gdf.drop(columns=['lat_fixed', 'lon_fixed', 'lat_numeric', 'lon_numeric'], inplace=True, errors='ignore')
    
else:
    print("⚠ Warning: No se pudo procesar el GeoDataFrame o faltan columnas lat/lon")

CORRIGIENDO FORMATO DE COORDENADAS

Corrigiendo valores de latitud...
Corrigiendo valores de longitud...

Recreando geometrías con coordenadas corregidas...

✓ CORRECCIÓN COMPLETADA
Registros totales: 1641
Coordenadas válidas después de corrección: 1560
Coordenadas inválidas: 81

Muestra de coordenadas corregidas:
    upid                                          nombre_up       lat  \
0  UNP-1                                   Misn Santa Elena  3.441883   
1  UNP-2  Asignación del Subsidio Distrital de Vivienda ...  3.327042   
2  UNP-3  Asignación del Subsidio Distrital de Vivienda ...  3.400405   
3  UNP-4  Asignación del Subsidio Distrital de Vivienda ...  3.341943   
4  UNP-5  Asignación del Subsidio Distrital de Vivienda ...  3.400405   

         lon  
0 -76.520562  
1 -76.503578  
2 -76.521555  
3 -76.510784  
4 -76.521555  


### 14.3. Crear el campo "geometry"

 Para las coordenadas en formato "lat, lon" tipo "POINT" compatible con el estandar geojson (pero "lat, "lon"), y el resto de datos como atributos, preparando los datos como geodataframe, después de crear "geometry", eliminar las columnas "lat" y "lon".

In [29]:
# Create final geometry column and prepare the GeoDataFrame
print("=" * 60)
print("CREANDO GEOMETRÍA FINAL Y PREPARANDO GEODATAFRAME")
print("=" * 60)
print()

if gdf is not None and 'lat' in gdf.columns and 'lon' in gdf.columns:
    # Ensure geometry column exists with valid coordinates
    valid_coords_mask = gdf['lat'].notna() & gdf['lon'].notna()
    
    # Create Point geometries (lat, lon order as requested)
    gdf.loc[valid_coords_mask, 'geometry'] = gdf.loc[valid_coords_mask].apply(
        lambda row: Point(row['lat'], row['lon']), 
        axis=1
    )
    
    # Convert to GeoDataFrame with proper CRS
    gdf = gpd.GeoDataFrame(gdf, geometry='geometry', crs='EPSG:4326')
    
    print(f"✓ Geometrías creadas: {gdf['geometry'].notna().sum()}")
    print(f"✓ CRS: {gdf.crs}")
    print(f"✓ Orden de coordenadas: lat, lon")
    print()
    
    # Display sample before dropping lat/lon
    print("Muestra de datos antes de eliminar lat/lon:")
    print(gdf[['upid', 'nombre_up', 'lat', 'lon', 'geometry']].head(3))
    print()
    
    # Drop lat and lon columns
    gdf.drop(columns=['lat', 'lon'], inplace=True)
    
    print("✓ Columnas 'lat' y 'lon' eliminadas")
    print()
    print(f"✓ Columnas finales en el GeoDataFrame: {len(gdf.columns)}")
    print()
    
    # Show final structure
    print("Estructura final del GeoDataFrame:")
    print(gdf[['upid', 'nombre_up', 'geometry']].head(3))
    print()
    
    # Display geometry info
    print("Información de geometrías:")
    print(f"  - Total de registros: {len(gdf)}")
    print(f"  - Geometrías válidas: {gdf['geometry'].notna().sum()}")
    print(f"  - Geometrías nulas: {gdf['geometry'].isna().sum()}")
    print(f"  - Tipo de geometría: Point")
    print(f"  - Formato: POINT(lat lon)")
    print()
    
    print("✓ GEODATAFRAME PREPARADO EXITOSAMENTE")
    print("=" * 60)
else:
    print("⚠ Error: No se pudo crear el GeoDataFrame - columnas lat/lon no disponibles")

CREANDO GEOMETRÍA FINAL Y PREPARANDO GEODATAFRAME

✓ Geometrías creadas: 1561
✓ CRS: EPSG:4326
✓ Orden de coordenadas: lat, lon

Muestra de datos antes de eliminar lat/lon:
    upid                                          nombre_up       lat  \
0  UNP-1                                   Misn Santa Elena  3.441883   
1  UNP-2  Asignación del Subsidio Distrital de Vivienda ...  3.327042   
2  UNP-3  Asignación del Subsidio Distrital de Vivienda ...  3.400405   

         lon                   geometry  
0 -76.520562  POINT (3.44188 -76.52056)  
1 -76.503578  POINT (3.32704 -76.50358)  
2 -76.521555   POINT (3.4004 -76.52156)  

✓ Columnas 'lat' y 'lon' eliminadas

✓ Columnas finales en el GeoDataFrame: 27

Estructura final del GeoDataFrame:
    upid                                          nombre_up  \
0  UNP-1                                   Misn Santa Elena   
1  UNP-2  Asignación del Subsidio Distrital de Vivienda ...   
2  UNP-3  Asignación del Subsidio Distrital de Vivienda ...  

### 14.4. Encontrar Coordenadas por fuera de los límites de Santiago de Cali, Colombia.

In [30]:
# Define Santiago de Cali approximate boundaries
CALI_LAT_MIN = 3.30
CALI_LAT_MAX = 3.60
CALI_LON_MIN = -76.60
CALI_LON_MAX = -76.45

print("=" * 60)
print("IDENTIFICANDO COORDENADAS FUERA DE SANTIAGO DE CALI")
print("=" * 60)
print()
print(f"Límites de Santiago de Cali:")
print(f"  Latitud: {CALI_LAT_MIN} a {CALI_LAT_MAX}")
print(f"  Longitud: {CALI_LON_MIN} a {CALI_LON_MAX}")
print()

if gdf is not None and 'geometry' in gdf.columns:
    # Extract lat/lon from geometry for records with valid geometry
    valid_geom_mask = gdf['geometry'].notna()
    
    # IMPORTANTE: geometry está en formato Point(lat, lon), no Point(lon, lat)
    # Por lo tanto: .x = lat, .y = lon
    lats = gdf[valid_geom_mask].geometry.x  # x contiene lat
    lons = gdf[valid_geom_mask].geometry.y  # y contiene lon
    
    # Find records outside Cali boundaries
    outside_bounds = (
        (lats < CALI_LAT_MIN) | 
        (lats > CALI_LAT_MAX) | 
        (lons < CALI_LON_MIN) | 
        (lons > CALI_LON_MAX)
    )
    
    # Filter records outside boundaries
    records_outside = gdf[valid_geom_mask][outside_bounds]
    
    print(f"✓ RESULTADOS:")
    print(f"  Total de registros con geometría: {valid_geom_mask.sum()}")
    print(f"  Registros fuera de límites: {len(records_outside)}")
    print(f"  Registros dentro de límites: {valid_geom_mask.sum() - len(records_outside)}")
    print()
    
    if len(records_outside) > 0:
        print("⚠ REGISTROS FUERA DE LOS LÍMITES DE CALI:")
        print("=" * 60)
        
        # Create summary dataframe with relevant columns
        outside_summary = records_outside.copy()
        outside_summary['lat_value'] = outside_summary.geometry.x  # x es lat
        outside_summary['lon_value'] = outside_summary.geometry.y  # y es lon
        
        # Display columns to show
        display_cols = ['upid', 'nombre_up', 'lat_value', 'lon_value', 'direccion', 'barrio_vereda']
        available_cols = [col for col in display_cols if col in outside_summary.columns]
        
        print(outside_summary[available_cols].to_string(index=False))
        print()
        
        # Show detailed statistics
        print("Estadísticas de coordenadas fuera de límites:")
        print(f"  Latitud mínima: {outside_summary['lat_value'].min():.6f}")
        print(f"  Latitud máxima: {outside_summary['lat_value'].max():.6f}")
        print(f"  Longitud mínima: {outside_summary['lon_value'].min():.6f}")
        print(f"  Longitud máxima: {outside_summary['lon_value'].max():.6f}")
        
    else:
        print("✓ Todas las coordenadas están dentro de los límites de Santiago de Cali")
        
else:
    print("⚠ Error: GeoDataFrame no disponible o sin columna 'geometry'")

IDENTIFICANDO COORDENADAS FUERA DE SANTIAGO DE CALI

Límites de Santiago de Cali:
  Latitud: 3.3 a 3.6
  Longitud: -76.6 a -76.45

✓ RESULTADOS:
  Total de registros con geometría: 1561
  Registros fuera de límites: 12
  Registros dentro de límites: 1549

⚠ REGISTROS FUERA DE LOS LÍMITES DE CALI:
    upid                                                               nombre_up  lat_value  lon_value                                  direccion                barrio_vereda
 UNP-678 Asignación del Subsidio Distrital de Vivienda para el Cierre Financiero  -7.650763   3.369127                            Cra 98c #5b8-72 Plan Parcial Ciudad Melendez
UNP-1286                                               Parque Principal Pichinde   3.437222 -76.613750             Corregimiento Pichinde (rural)                     CABECERA
UNP-1287                                              Graderia Cancha la Leonera   3.455630 -76.636189           Corregimiento la Leonera (rural)                     CABECERA
UN

### 14.5. Realizar una intersección de capas de gdf, con basemaps

In [31]:
import json

# Load the barrios_veredas GeoJSON file
barrios_veredas_path = workspace_path / 'basemaps' / 'barrios_veredas.geojson'

print("=" * 60)
print("REALIZANDO INTERSECCIÓN CON BARRIOS Y VEREDAS")
print("=" * 60)
print()

if barrios_veredas_path.exists():
    # Load the barrios_veredas GeoJSON
    barrios_veredas_gdf = gpd.read_file(barrios_veredas_path)
    
    print(f"✓ Archivo cargado: {barrios_veredas_path.name}")
    print(f"  - Registros en barrios_veredas: {len(barrios_veredas_gdf)}")
    print(f"  - CRS: {barrios_veredas_gdf.crs}")
    print()
    
    # Ensure both GeoDataFrames have the same CRS
    if gdf.crs != barrios_veredas_gdf.crs:
        print(f"⚠ Ajustando CRS de barrios_veredas de {barrios_veredas_gdf.crs} a {gdf.crs}")
        barrios_veredas_gdf = barrios_veredas_gdf.to_crs(gdf.crs)
    
    # IMPORTANT: Since geometry in gdf is Point(lat, lon) instead of Point(lon, lat),
    # we need to swap coordinates before the spatial join
    print("Preparando geometrías para la intersección...")
    gdf_temp = gdf.copy()
    
    # Identify records with valid geometry
    valid_geom = gdf_temp['geometry'].notna()
    print(f"  - Registros con geometría válida: {valid_geom.sum()}")
    print(f"  - Registros sin geometría: {(~valid_geom).sum()}")
    print()
    
    # Create proper Point geometries with lon, lat order for spatial operations
    # Only for records with valid geometry
    gdf_temp.loc[valid_geom, 'geometry'] = gdf_temp.loc[valid_geom, 'geometry'].apply(
        lambda geom: Point(geom.y, geom.x) if geom else None  # Swap: y=lon, x=lat -> Point(lon, lat)
    )
    
    print("Realizando intersección espacial...")
    gdf_with_barrio = gpd.sjoin(
        gdf_temp, 
        barrios_veredas_gdf[['geometry', 'barrio_vereda']], 
        how='left', 
        predicate='within'
    )
    
    # Extract the barrio_vereda_2 column from the spatial join result
    gdf['barrio_vereda_2'] = gdf_with_barrio['barrio_vereda_right'] if 'barrio_vereda_right' in gdf_with_barrio.columns else gdf_with_barrio.get('barrio_vereda', None)
    
    # Clean up index_right column if it exists
    if 'index_right' in gdf.columns:
        gdf.drop(columns=['index_right'], inplace=True)
    
    print()
    print("✓ INTERSECCIÓN COMPLETADA")
    print("=" * 60)
    print(f"Registros totales: {len(gdf)}")
    print(f"Registros con geometría válida: {valid_geom.sum()}")
    print(f"Registros con barrio_vereda_2 asignado: {gdf['barrio_vereda_2'].notna().sum()}")
    print(f"Registros sin barrio_vereda_2: {gdf['barrio_vereda_2'].isna().sum()}")
    print()
    
    # Show sample of results
    print("Muestra de resultados:")
    display_cols = ['upid', 'nombre_up', 'barrio_vereda', 'barrio_vereda_2']
    available_cols = [col for col in display_cols if col in gdf.columns]
    print(gdf[gdf['barrio_vereda_2'].notna()][available_cols].head(10))
    
    # Show comparison statistics if original barrio_vereda exists
    if 'barrio_vereda' in gdf.columns:
        print()
        print("Comparación de barrio_vereda original vs barrio_vereda_2:")
        both_valid = gdf['barrio_vereda'].notna() & gdf['barrio_vereda_2'].notna()
        matches = (gdf.loc[both_valid, 'barrio_vereda'] == gdf.loc[both_valid, 'barrio_vereda_2']).sum()
        print(f"  - Registros con ambos valores: {both_valid.sum()}")
        print(f"  - Coincidencias: {matches}")
        print(f"  - Diferencias: {both_valid.sum() - matches}")
    
else:
    print(f"⚠ Error: No se encontró el archivo {barrios_veredas_path}")
    print("   Verifica que el archivo 'barrios_veredas.geojson' exista en la carpeta 'basemaps'")


REALIZANDO INTERSECCIÓN CON BARRIOS Y VEREDAS

✓ Archivo cargado: barrios_veredas.geojson
  - Registros en barrios_veredas: 433
  - CRS: EPSG:4326

Preparando geometrías para la intersección...
  - Registros con geometría válida: 1561
  - Registros sin geometría: 80

Realizando intersección espacial...

✓ INTERSECCIÓN COMPLETADA
Registros totales: 1641
Registros con geometría válida: 1561
Registros con barrio_vereda_2 asignado: 1179
Registros sin barrio_vereda_2: 462

Muestra de resultados:
      upid                                          nombre_up  \
0    UNP-1                                   Misn Santa Elena   
2    UNP-3  Asignación del Subsidio Distrital de Vivienda ...   
4    UNP-5  Asignación del Subsidio Distrital de Vivienda ...   
5    UNP-6  Asignación del Subsidio Distrital de Vivienda ...   
6    UNP-7  Asignación del Subsidio Distrital de Vivienda ...   
7    UNP-8  Asignación del Subsidio Distrital de Vivienda ...   
8    UNP-9  Asignación del Subsidio Distrital de 

In [32]:
# Load the comunas_corregimientos GeoJSON file
comunas_corregimientos_path = workspace_path / 'basemaps' / 'comunas_corregimientos.geojson'

print("=" * 60)
print("REALIZANDO INTERSECCIÓN CON COMUNAS Y CORREGIMIENTOS")
print("=" * 60)
print()

if comunas_corregimientos_path.exists():
    # Load the comunas_corregimientos GeoJSON
    comunas_corregimientos_gdf = gpd.read_file(comunas_corregimientos_path)
    
    print(f"✓ Archivo cargado: {comunas_corregimientos_path.name}")
    print(f"  - Registros en comunas_corregimientos: {len(comunas_corregimientos_gdf)}")
    print(f"  - CRS: {comunas_corregimientos_gdf.crs}")
    print()
    
    # Ensure both GeoDataFrames have the same CRS
    if gdf.crs != comunas_corregimientos_gdf.crs:
        print(f"⚠ Ajustando CRS de comunas_corregimientos de {comunas_corregimientos_gdf.crs} a {gdf.crs}")
        comunas_corregimientos_gdf = comunas_corregimientos_gdf.to_crs(gdf.crs)
    
    # Prepare geometries for spatial operations
    print("Preparando geometrías para la intersección...")
    gdf_temp = gdf.copy()
    
    # Identify records with valid geometry
    valid_geom = gdf_temp['geometry'].notna()
    print(f"  - Registros con geometría válida: {valid_geom.sum()}")
    print(f"  - Registros sin geometría: {(~valid_geom).sum()}")
    print()
    
    # Create proper Point geometries with lon, lat order for spatial operations
    gdf_temp.loc[valid_geom, 'geometry'] = gdf_temp.loc[valid_geom, 'geometry'].apply(
        lambda geom: Point(geom.y, geom.x) if geom else None  # Swap: y=lon, x=lat -> Point(lon, lat)
    )
    
    print("Realizando intersección espacial...")
    gdf_with_comuna = gpd.sjoin(
        gdf_temp, 
        comunas_corregimientos_gdf[['geometry', 'comuna_corregimiento']], 
        how='left', 
        predicate='within'
    )
    
    # Extract the comuna_corregimiento_2 column from the spatial join result
    gdf['comuna_corregimiento_2'] = gdf_with_comuna['comuna_corregimiento_right'] if 'comuna_corregimiento_right' in gdf_with_comuna.columns else gdf_with_comuna.get('comuna_corregimiento', None)
    
    # Clean up index_right column if it exists
    if 'index_right' in gdf.columns:
        gdf.drop(columns=['index_right'], inplace=True)
    
    print()
    print("✓ INTERSECCIÓN COMPLETADA")
    print("=" * 60)
    print(f"Registros totales: {len(gdf)}")
    print(f"Registros con geometría válida: {valid_geom.sum()}")
    print(f"Registros con comuna_corregimiento_2 asignado: {gdf['comuna_corregimiento_2'].notna().sum()}")
    print(f"Registros sin comuna_corregimiento_2: {gdf['comuna_corregimiento_2'].isna().sum()}")
    print()
    
    # Show sample of results
    print("Muestra de resultados:")
    display_cols = ['upid', 'nombre_up', 'comuna_corregimiento', 'comuna_corregimiento_2']
    available_cols = [col for col in display_cols if col in gdf.columns]
    print(gdf[gdf['comuna_corregimiento_2'].notna()][available_cols].head(10))
    
    # Show comparison statistics if original comuna_corregimiento exists
    if 'comuna_corregimiento' in gdf.columns:
        print()
        print("Comparación de comuna_corregimiento original vs comuna_corregimiento_2:")
        both_valid = gdf['comuna_corregimiento'].notna() & gdf['comuna_corregimiento_2'].notna()
        matches = (gdf.loc[both_valid, 'comuna_corregimiento'] == gdf.loc[both_valid, 'comuna_corregimiento_2']).sum()
        print(f"  - Registros con ambos valores: {both_valid.sum()}")
        print(f"  - Coincidencias: {matches}")
        print(f"  - Diferencias: {both_valid.sum() - matches}")
    
else:
    print(f"⚠ Error: No se encontró el archivo {comunas_corregimientos_path}")
    print("   Verifica que el archivo 'comunas_corregimientos.geojson' exista en la carpeta 'basemaps'")


REALIZANDO INTERSECCIÓN CON COMUNAS Y CORREGIMIENTOS

✓ Archivo cargado: comunas_corregimientos.geojson
  - Registros en comunas_corregimientos: 37
  - CRS: EPSG:4326

Preparando geometrías para la intersección...
  - Registros con geometría válida: 1561
  - Registros sin geometría: 80

Realizando intersección espacial...

✓ INTERSECCIÓN COMPLETADA
Registros totales: 1641
Registros con geometría válida: 1561
Registros con comuna_corregimiento_2 asignado: 1180
Registros sin comuna_corregimiento_2: 461

Muestra de resultados:
      upid                                          nombre_up  \
0    UNP-1                                   Misn Santa Elena   
2    UNP-3  Asignación del Subsidio Distrital de Vivienda ...   
4    UNP-5  Asignación del Subsidio Distrital de Vivienda ...   
5    UNP-6  Asignación del Subsidio Distrital de Vivienda ...   
6    UNP-7  Asignación del Subsidio Distrital de Vivienda ...   
7    UNP-8  Asignación del Subsidio Distrital de Vivienda ...   
8    UNP-9  Asi

In [33]:
from difflib import get_close_matches
import unicodedata
import pandas as pd
import geopandas as gpd

def normalize_text(text):
    """
    Normalize text by removing accents, converting to uppercase, and standardizing whitespace.
    
    Args:
        text: String to normalize
        
    Returns:
        Normalized string
    """
    if pd.isna(text) or text is None:
        return ""
    
    # Convert to string and strip
    text = str(text).strip()
    
    # Remove accents/diacritics
    text = unicodedata.normalize('NFD', text)
    text = ''.join(char for char in text if unicodedata.category(char) != 'Mn')
    
    # Convert to uppercase and normalize whitespace
    text = ' '.join(text.upper().split())
    
    return text


def normalize_comuna_value(value):
    """
    Normalize comuna values to standard format (COMUNA 01, COMUNA 02, etc.)
    Values > 22 are set to "RURAL"
    
    Args:
        value: Comuna value to normalize
        
    Returns:
        Normalized comuna value
    """
    if pd.isna(value) or value is None or value == "":
        return None
    
    text = str(value).strip().upper()
    
    # Check if it matches pattern "COMUNA X" or "COMUNA XX"
    if text.startswith("COMUNA"):
        parts = text.split()
        if len(parts) >= 2:
            try:
                num = int(parts[1])
                if num > 22:
                    return "RURAL"
                elif num < 10:
                    return f"COMUNA {num:02d}"
                else:
                    return f"COMUNA {num}"
            except ValueError:
                pass
    
    return value


def find_best_match(value, standard_values, threshold=0.6):
    """
    Find the best matching value from a list of standard values.
    
    Args:
        value: Value to match
        standard_values: List of standard values to match against
        threshold: Minimum similarity score (0-1)
        
    Returns:
        Best matching value or None if no good match found
    """
    if pd.isna(value) or value is None or value == "":
        return None
    
    # Normalize the input value
    normalized_value = normalize_text(value)
    
    # Normalize all standard values
    normalized_standards = {normalize_text(std): std for std in standard_values if pd.notna(std)}
    
    # Find close matches
    matches = get_close_matches(normalized_value, normalized_standards.keys(), n=1, cutoff=threshold)
    
    if matches:
        # Return the original (non-normalized) standard value
        return normalized_standards[matches[0]]
    
    return None


print("=" * 60)
print("NORMALIZANDO VALORES DE COMUNA/CORREGIMIENTO Y BARRIO/VEREDA")
print("=" * 60)
print()

# Get standard values from the geojson files
if 'barrios_veredas_gdf' in locals() and barrios_veredas_gdf is not None:
    standard_barrios = barrios_veredas_gdf['barrio_vereda'].dropna().unique().tolist()
    print(f"✓ Valores estándar de barrios/veredas cargados: {len(standard_barrios)}")
else:
    print("⚠ Warning: No se encontró barrios_veredas_gdf, cargando desde archivo...")
    barrios_veredas_path = workspace_path / 'basemaps' / 'barrios_veredas.geojson'
    if barrios_veredas_path.exists():
        barrios_veredas_gdf = gpd.read_file(barrios_veredas_path)
        standard_barrios = barrios_veredas_gdf['barrio_vereda'].dropna().unique().tolist()
        print(f"✓ Valores estándar de barrios/veredas cargados: {len(standard_barrios)}")
    else:
        print("⚠ Error: No se pudo cargar barrios_veredas.geojson")
        standard_barrios = []

if 'comunas_corregimientos_gdf' in locals() and comunas_corregimientos_gdf is not None:
    standard_comunas = comunas_corregimientos_gdf['comuna_corregimiento'].dropna().unique().tolist()
    print(f"✓ Valores estándar de comunas/corregimientos cargados: {len(standard_comunas)}")
else:
    print("⚠ Warning: No se encontró comunas_corregimientos_gdf, cargando desde archivo...")
    comunas_corregimientos_path = workspace_path / 'basemaps' / 'comunas_corregimientos.geojson'
    if comunas_corregimientos_path.exists():
        comunas_corregimientos_gdf = gpd.read_file(comunas_corregimientos_path)
        standard_comunas = comunas_corregimientos_gdf['comuna_corregimiento'].dropna().unique().tolist()
        print(f"✓ Valores estándar de comunas/corregimientos cargados: {len(standard_comunas)}")
    else:
        print("⚠ Error: No se pudo cargar comunas_corregimientos.geojson")
        standard_comunas = []

print()

# Normalize comuna_corregimiento values
changes_comuna = []
if 'comuna_corregimiento' in gdf.columns and len(standard_comunas) > 0:
    print("Normalizando valores de comuna_corregimiento...")
    
    for idx in gdf.index:
        original_value = gdf.at[idx, 'comuna_corregimiento']
        
        if pd.notna(original_value):
            # First normalize COMUNA format
            normalized_comuna = normalize_comuna_value(original_value)
            
            # Then find best match if needed
            if normalized_comuna != original_value:
                changes_comuna.append({
                    'upid': gdf.at[idx, 'upid'],
                    'original': original_value,
                    'normalized': normalized_comuna
                })
                gdf.at[idx, 'comuna_corregimiento'] = normalized_comuna
            else:
                # Try fuzzy matching
                best_match = find_best_match(normalized_comuna, standard_comunas, threshold=0.7)
                
                if best_match and best_match != original_value:
                    changes_comuna.append({
                        'upid': gdf.at[idx, 'upid'],
                        'original': original_value,
                        'normalized': best_match
                    })
                    gdf.at[idx, 'comuna_corregimiento'] = best_match
    
    print(f"  ✓ Valores normalizados: {len(changes_comuna)}")
    
    if changes_comuna:
        print(f"\n  Muestra de cambios (primeros 10):")
        for i, change in enumerate(changes_comuna[:10]):
            print(f"    {change['upid']}: '{change['original']}' → '{change['normalized']}'")

print()

# Normalize barrio_vereda values
changes_barrio = []
if 'barrio_vereda' in gdf.columns and len(standard_barrios) > 0:
    print("Normalizando valores de barrio_vereda...")
    
    for idx in gdf.index:
        original_value = gdf.at[idx, 'barrio_vereda']
        
        if pd.notna(original_value):
            best_match = find_best_match(original_value, standard_barrios, threshold=0.7)
            
            if best_match and best_match != original_value:
                changes_barrio.append({
                    'upid': gdf.at[idx, 'upid'],
                    'original': original_value,
                    'normalized': best_match
                })
                gdf.at[idx, 'barrio_vereda'] = best_match
    
    print(f"  ✓ Valores normalizados: {len(changes_barrio)}")
    
    if changes_barrio:
        print(f"\n  Muestra de cambios (primeros 10):")
        for i, change in enumerate(changes_barrio[:10]):
            print(f"    {change['upid']}: '{change['original']}' → '{change['normalized']}'")

print()
print("=" * 60)
print("✓ NORMALIZACIÓN COMPLETADA")
print("=" * 60)
print()

# Show summary statistics
print("Estadísticas finales:")
print(f"  Comuna/Corregimiento:")
print(f"    - Valores únicos: {gdf['comuna_corregimiento'].nunique() if 'comuna_corregimiento' in gdf.columns else 0}")
print(f"    - Valores normalizados: {len(changes_comuna)}")

print(f"  Barrio/Vereda:")
print(f"    - Valores únicos: {gdf['barrio_vereda'].nunique() if 'barrio_vereda' in gdf.columns else 0}")
print(f"    - Valores normalizados: {len(changes_barrio)}")


NORMALIZANDO VALORES DE COMUNA/CORREGIMIENTO Y BARRIO/VEREDA

✓ Valores estándar de barrios/veredas cargados: 423
✓ Valores estándar de comunas/corregimientos cargados: 37

Normalizando valores de comuna_corregimiento...
  ✓ Valores normalizados: 214

  Muestra de cambios (primeros 10):
    UNP-15: 'COMUNA 8' → 'COMUNA 08'
    UNP-31: 'COMUNA 8' → 'COMUNA 08'
    UNP-36: 'COMUNA 8' → 'COMUNA 08'
    UNP-58: 'COMUNA 8' → 'COMUNA 08'
    UNP-59: 'COMUNA 8' → 'COMUNA 08'
    UNP-342: 'COMUNA 7' → 'COMUNA 07'
    UNP-343: 'COMUNA 8' → 'COMUNA 08'
    UNP-345: 'COMUNA 8' → 'COMUNA 08'
    UNP-346: 'COMUNA 8' → 'COMUNA 08'
    UNP-349: 'COMUNA 8' → 'COMUNA 08'

Normalizando valores de barrio_vereda...
  ✓ Valores normalizados: 557

  Muestra de cambios (primeros 10):
    UNP-1: 'Polvorines ' → 'Polvorines'
    UNP-17: 'Ricardo Balcazar' → 'Ricardo Balcázar'
    UNP-25: 'Ricardo Balcazar' → 'Ricardo Balcázar'
    UNP-49: 'Ricardo Balcazar' → 'Ricardo Balcázar'
    UNP-51: 'Ricardo Balcazar' →

In [34]:
# Create "fuera_rango" validation column
print("=" * 60)
print("CREANDO COLUMNA DE VALIDACIÓN 'fuera_rango'")
print("=" * 60)
print()

if gdf is not None and 'comuna_corregimiento_2' in gdf.columns and 'barrio_vereda_2' in gdf.columns:
    
    # Initialize the fuera_rango column
    gdf['fuera_rango'] = None
    
    # Get records with valid geometry
    valid_geom_mask = gdf['geometry'].notna()
    
    print(f"Registros con geometría válida: {valid_geom_mask.sum()}")
    print(f"Registros sin geometría: {(~valid_geom_mask).sum()}")
    print()
    
    # Check if comuna_corregimiento matches
    comuna_matches = (
        gdf['comuna_corregimiento'].notna() & 
        gdf['comuna_corregimiento_2'].notna() &
        (gdf['comuna_corregimiento'] == gdf['comuna_corregimiento_2'])
    )
    
    # Check if barrio_vereda matches
    barrio_matches = (
        gdf['barrio_vereda'].notna() & 
        gdf['barrio_vereda_2'].notna() &
        (gdf['barrio_vereda'] == gdf['barrio_vereda_2'])
    )
    
    # Records are ACEPTABLE if:
    # 1. They have valid geometry AND
    # 2. Either comuna_corregimiento matches OR barrio_vereda matches (or both)
    aceptable_mask = valid_geom_mask & (comuna_matches | barrio_matches)
    
    # Records are FUERA DE RANGO if:
    # 1. They have valid geometry AND
    # 2. Neither comuna_corregimiento nor barrio_vereda match
    fuera_rango_mask = valid_geom_mask & ~(comuna_matches | barrio_matches)
    
    # Assign values
    gdf.loc[aceptable_mask, 'fuera_rango'] = 'ACEPTABLE'
    gdf.loc[fuera_rango_mask, 'fuera_rango'] = 'FUERA DE RANGO'
    
    # Records without geometry remain None
    
    print("✓ VALIDACIÓN COMPLETADA")
    print("=" * 60)
    print(f"Total de registros: {len(gdf)}")
    print(f"ACEPTABLE: {(gdf['fuera_rango'] == 'ACEPTABLE').sum()}")
    print(f"FUERA DE RANGO: {(gdf['fuera_rango'] == 'FUERA DE RANGO').sum()}")
    print(f"Sin geometría (None): {gdf['fuera_rango'].isna().sum()}")
    print()
    
    # Show detailed breakdown
    print("Desglose de validación:")
    print(f"  - Comuna coincide: {comuna_matches.sum()}")
    print(f"  - Barrio coincide: {barrio_matches.sum()}")
    print(f"  - Ambos coinciden: {(comuna_matches & barrio_matches).sum()}")
    print(f"  - Solo comuna coincide: {(comuna_matches & ~barrio_matches).sum()}")
    print(f"  - Solo barrio coincide: {(barrio_matches & ~comuna_matches).sum()}")
    print(f"  - Ninguno coincide: {(~comuna_matches & ~barrio_matches & valid_geom_mask).sum()}")
    print()
    
    # Show sample of FUERA DE RANGO records
    if fuera_rango_mask.sum() > 0:
        print("Muestra de registros FUERA DE RANGO:")
        print("=" * 60)
        display_cols = [
            'upid', 'nombre_up', 
            'comuna_corregimiento', 'comuna_corregimiento_2',
            'barrio_vereda', 'barrio_vereda_2',
            'fuera_rango'
        ]
        available_cols = [col for col in display_cols if col in gdf.columns]
        print(gdf[fuera_rango_mask][available_cols].head(10))
    else:
        print("✓ No hay registros FUERA DE RANGO")
    
else:
    print("⚠ Error: No se pueden realizar validaciones - columnas necesarias no disponibles")

CREANDO COLUMNA DE VALIDACIÓN 'fuera_rango'

Registros con geometría válida: 1561
Registros sin geometría: 80

✓ VALIDACIÓN COMPLETADA
Total de registros: 1641
ACEPTABLE: 1111
FUERA DE RANGO: 450
Sin geometría (None): 80

Desglose de validación:
  - Comuna coincide: 1106
  - Barrio coincide: 789
  - Ambos coinciden: 784
  - Solo comuna coincide: 322
  - Solo barrio coincide: 5
  - Ninguno coincide: 450

Muestra de registros FUERA DE RANGO:
      upid                                          nombre_up  \
0    UNP-1                                   Misn Santa Elena   
1    UNP-2  Asignación del Subsidio Distrital de Vivienda ...   
3    UNP-4  Asignación del Subsidio Distrital de Vivienda ...   
9   UNP-10  Asignación del Subsidio Distrital de Vivienda ...   
10  UNP-11  Asignación del Subsidio Distrital de Vivienda ...   
11  UNP-12  Asignación del Subsidio Distrital de Vivienda ...   
12  UNP-13  Asignación del Subsidio Distrital de Vivienda ...   
13  UNP-14  Asignación del Subsidio 

### 14.6. Revisión de fechas


In [35]:
import pandas as pd
from datetime import datetime
import re
from datetime import timedelta

print("=" * 60)
print("REVISIÓN Y ESTANDARIZACIÓN DE FECHAS")
print("=" * 60)
print()

# Function to parse and standardize date values
def parse_date(date_value):
    """
    Parse various date formats and return standardized datetime or None.
    
    Handles formats:
    - DD/MM/YYYY
    - DD-MM-YYYY
    - YYYY/MM/DD
    - YYYY-MM-DD
    - Excel serial dates (numeric)
    - ISO format dates
    """
    if pd.isna(date_value) or date_value is None:
        return None
    
    # If already datetime, return as is
    if isinstance(date_value, datetime):
        return date_value
    
    # Convert to string for processing
    date_str = str(date_value).strip()
    
    # Handle empty strings
    if date_str == '' or date_str.lower() in ['nan', 'none', 'null']:
        return None
    
    # Try parsing Excel serial date (numeric values like 45896)
    try:
        date_num = float(date_str)
        if 40000 <= date_num <= 60000:  # Reasonable range for Excel dates (2009-2064)
            # Excel epoch: December 30, 1899
            excel_epoch = datetime(1899, 12, 30)
            return excel_epoch + timedelta(days=date_num)
    except (ValueError, TypeError):
        pass
    
    # List of date format patterns to try
    date_patterns = [
        r'(\d{2})/(\d{2})/(\d{4})',      # DD/MM/YYYY
        r'(\d{2})-(\d{2})-(\d{4})',      # DD-MM-YYYY
        r'(\d{4})/(\d{2})/(\d{2})',      # YYYY/MM/DD
        r'(\d{4})-(\d{2})-(\d{2})',      # YYYY-MM-DD
        r'(\d{1,2})-([a-zA-Z]{3})-(\d{2})',  # D-MMM-YY (e.g., 12-nov-24)
    ]
    
    for pattern in date_patterns:
        match = re.search(pattern, date_str)
        if match:
            groups = match.groups()
            
            try:
                # Handle D-MMM-YY format
                if len(groups) == 3 and not groups[1].isdigit():
                    day = int(groups[0])
                    month_str = groups[1].lower()
                    year = int(groups[2])
                    
                    # Convert month abbreviation to number
                    months_es = {
                        'ene': 1, 'feb': 2, 'mar': 3, 'abr': 4, 'may': 5, 'jun': 6,
                        'jul': 7, 'ago': 8, 'sep': 9, 'oct': 10, 'nov': 11, 'dic': 12
                    }
                    months_en = {
                        'jan': 1, 'feb': 2, 'mar': 3, 'apr': 4, 'may': 5, 'jun': 6,
                        'jul': 7, 'aug': 8, 'sep': 9, 'oct': 10, 'nov': 11, 'dec': 12
                    }
                    
                    month = months_es.get(month_str) or months_en.get(month_str)
                    if not month:
                        continue
                    
                    # Handle 2-digit year
                    if year < 100:
                        year = 2000 + year if year < 50 else 1900 + year
                    
                    return datetime(year, month, day)
                
                # Handle numeric date patterns
                elif len(groups) == 3 and all(g.isdigit() for g in groups):
                    # Determine if it's DD/MM/YYYY or YYYY/MM/DD based on first number
                    if len(groups[0]) == 4:  # YYYY/MM/DD
                        year, month, day = int(groups[0]), int(groups[1]), int(groups[2])
                    else:  # DD/MM/YYYY
                        day, month, year = int(groups[0]), int(groups[1]), int(groups[2])
                    
                    # Validate ranges
                    if 1 <= day <= 31 and 1 <= month <= 12 and 1900 <= year <= 2100:
                        return datetime(year, month, day)
                        
            except (ValueError, TypeError):
                continue
    
    # If no pattern matched, try pandas to_datetime as last resort
    try:
        return pd.to_datetime(date_str, errors='coerce')
    except:
        return None


# Analyze current date formats
print("ANÁLISIS DE FORMATOS DE FECHA ACTUALES")
print("-" * 60)

if 'fecha_inicio' in gdf.columns:
    print("\nfecha_inicio:")
    fecha_inicio_samples = gdf['fecha_inicio'].dropna().head(10)
    for idx, val in fecha_inicio_samples.items():
        print(f"  {val} (tipo: {type(val).__name__})")
    
    print(f"\n  Total valores no nulos: {gdf['fecha_inicio'].notna().sum()}")
    print(f"  Total valores nulos: {gdf['fecha_inicio'].isna().sum()}")

if 'fecha_fin' in gdf.columns:
    print("\nfecha_fin:")
    fecha_fin_samples = gdf['fecha_fin'].dropna().head(10)
    for idx, val in fecha_fin_samples.items():
        print(f"  {val} (tipo: {type(val).__name__})")
    
    print(f"\n  Total valores no nulos: {gdf['fecha_fin'].notna().sum()}")
    print(f"  Total valores nulos: {gdf['fecha_fin'].isna().sum()}")

print()
print("=" * 60)
print("ESTANDARIZANDO FECHAS")
print("=" * 60)
print()

# Standardize fecha_inicio
if 'fecha_inicio' in gdf.columns:
    print("Procesando fecha_inicio...")
    gdf['fecha_inicio_std'] = gdf['fecha_inicio'].apply(parse_date)
    
    success_rate = gdf['fecha_inicio_std'].notna().sum() / gdf['fecha_inicio'].notna().sum() * 100
    print(f"  ✓ Conversiones exitosas: {gdf['fecha_inicio_std'].notna().sum()} / {gdf['fecha_inicio'].notna().sum()} ({success_rate:.1f}%)")
    
    failed_conversions = gdf[gdf['fecha_inicio'].notna() & gdf['fecha_inicio_std'].isna()]
    if len(failed_conversions) > 0:
        print(f"  ⚠ Conversiones fallidas: {len(failed_conversions)}")
        print("\n  Valores que no se pudieron convertir:")
        for idx, row in failed_conversions[['upid', 'fecha_inicio']].head(5).iterrows():
            print(f"    {row['upid']}: '{row['fecha_inicio']}'")

print()

# Standardize fecha_fin
if 'fecha_fin' in gdf.columns:
    print("Procesando fecha_fin...")
    gdf['fecha_fin_std'] = gdf['fecha_fin'].apply(parse_date)
    
    success_rate = gdf['fecha_fin_std'].notna().sum() / gdf['fecha_fin'].notna().sum() * 100
    print(f"  ✓ Conversiones exitosas: {gdf['fecha_fin_std'].notna().sum()} / {gdf['fecha_fin'].notna().sum()} ({success_rate:.1f}%)")
    
    failed_conversions = gdf[gdf['fecha_fin'].notna() & gdf['fecha_fin_std'].isna()]
    if len(failed_conversions) > 0:
        print(f"  ⚠ Conversiones fallidas: {len(failed_conversions)}")
        print("\n  Valores que no se pudieron convertir:")
        for idx, row in failed_conversions[['upid', 'fecha_fin']].head(5).iterrows():
            print(f"    {row['upid']}: '{row['fecha_fin']}'")

print()
print("=" * 60)
print("VALIDACIÓN DE FECHAS ESTANDARIZADAS")
print("=" * 60)
print()

# Validate date ranges
if 'fecha_inicio_std' in gdf.columns and 'fecha_fin_std' in gdf.columns:
    # Check for dates outside reasonable range (2020-2030)
    min_valid_date = datetime(2020, 1, 1)
    max_valid_date = datetime(2030, 12, 31)
    
    invalid_inicio = gdf[
        gdf['fecha_inicio_std'].notna() & 
        ((gdf['fecha_inicio_std'] < min_valid_date) | (gdf['fecha_inicio_std'] > max_valid_date))
    ]
    
    invalid_fin = gdf[
        gdf['fecha_fin_std'].notna() & 
        ((gdf['fecha_fin_std'] < min_valid_date) | (gdf['fecha_fin_std'] > max_valid_date))
    ]
    
    print(f"Fechas fuera de rango válido (2020-2030):")
    print(f"  fecha_inicio: {len(invalid_inicio)}")
    print(f"  fecha_fin: {len(invalid_fin)}")
    
    # Check for fecha_fin before fecha_inicio
    invalid_order = gdf[
        gdf['fecha_inicio_std'].notna() & 
        gdf['fecha_fin_std'].notna() & 
        (gdf['fecha_fin_std'] < gdf['fecha_inicio_std'])
    ]
    
    print(f"\nProyectos con fecha_fin anterior a fecha_inicio: {len(invalid_order)}")
    if len(invalid_order) > 0:
        print("\n  Muestra:")
        for idx, row in invalid_order[['upid', 'nombre_up', 'fecha_inicio_std', 'fecha_fin_std']].head(5).iterrows():
            print(f"    {row['upid']}: {row['fecha_inicio_std'].date()} → {row['fecha_fin_std'].date()}")

print()
print("ESTADÍSTICAS FINALES:")
print("-" * 60)
print(f"Total de registros: {len(gdf)}")
print(f"\nfecha_inicio_std:")
print(f"  Valores válidos: {gdf['fecha_inicio_std'].notna().sum()}")
print(f"  Valores nulos: {gdf['fecha_inicio_std'].isna().sum()}")
if gdf['fecha_inicio_std'].notna().sum() > 0:
    print(f"  Rango: {gdf['fecha_inicio_std'].min().date()} a {gdf['fecha_inicio_std'].max().date()}")

print(f"\nfecha_fin_std:")
print(f"  Valores válidos: {gdf['fecha_fin_std'].notna().sum()}")
print(f"  Valores nulos: {gdf['fecha_fin_std'].isna().sum()}")
if gdf['fecha_fin_std'].notna().sum() > 0:
    print(f"  Rango: {gdf['fecha_fin_std'].min().date()} a {gdf['fecha_fin_std'].max().date()}")

print()
print("✓ ESTANDARIZACIÓN DE FECHAS COMPLETADA")
print("=" * 60)

REVISIÓN Y ESTANDARIZACIÓN DE FECHAS

ANÁLISIS DE FORMATOS DE FECHA ACTUALES
------------------------------------------------------------

fecha_inicio:
  03-08-2025 (tipo: str)
  24-06-2025 (tipo: str)
  24-06-2025 (tipo: str)
  24-06-2025 (tipo: str)
  24-06-2025 (tipo: str)
  24-06-2025 (tipo: str)
  24-06-2025 (tipo: str)
  24-06-2025 (tipo: str)
  24-06-2025 (tipo: str)
  24-06-2025 (tipo: str)

  Total valores no nulos: 1382
  Total valores nulos: 259

fecha_fin:
  2025-10-30 00:00:00 (tipo: Timestamp)
  2025-11-30 00:00:00 (tipo: Timestamp)
  2025-11-30 00:00:00 (tipo: Timestamp)
  2025-11-30 00:00:00 (tipo: Timestamp)
  2025-11-30 00:00:00 (tipo: Timestamp)
  2025-11-30 00:00:00 (tipo: Timestamp)
  2025-11-30 00:00:00 (tipo: Timestamp)
  2025-11-30 00:00:00 (tipo: Timestamp)
  2025-11-30 00:00:00 (tipo: Timestamp)
  2025-11-30 00:00:00 (tipo: Timestamp)

  Total valores no nulos: 1394
  Total valores nulos: 247

ESTANDARIZANDO FECHAS

Procesando fecha_inicio...
  ✓ Conversiones

In [36]:
gdf

,referencia_proceso,referencia_contrato,bpin,identificador,tipo_equipamiento,fuente_financiacion,nombre_up,nombre_up_detalle,comuna_corregimiento,tipo_intervencion,...,descripcion_intervencion,nombre_centro_gestor,processed_timestamp,upid,geometry,barrio_vereda_2,comuna_corregimiento_2,fuera_rango,fecha_inicio_std,fecha_fin_std
0,None,None,2024760010165,Misn Santa Elena,Vivienda Nueva,Empréstito,Misn Santa Elena,Aportes Al Macrorpoyecto de Interes Social Nac...,COMUNA 18,Obra nueva,...,None,Secretaría de Vivienda Social y Habitat,2025-11-18T00:50:28.595762,UNP-1,POINT (3.44188 -76.52056),Belalcázar,COMUNA 09,FUERA DE RANGO,2025-08-03,2025-10-30
1,4244.0.9.10.341-2025,None,2024760010165,Subsidios para Cierre Financiero,Vivienda Nueva,Empréstito,Asignación del Subsidio Distrital de Vivienda ...,Asignación del Subsidio Distrital de Vivienda ...,EXPANSION URBANA,Obra nueva,...,None,Secretaría de Vivienda Social y Habitat,2025-11-18T00:50:28.595762,UNP-2,POINT (3.32704 -76.50358),NaN,NaN,FUERA DE RANGO,2025-06-24,2025-11-30
2,4244.0.9.10.341-2025,None,2024760010165,Subsidios para Cierre Financiero,Vivienda Nueva,Empréstito,Asignación del Subsidio Distrital de Vivienda ...,Asignación del Subsidio Distrital de Vivienda ...,COMUNA 16,Obra nueva,...,None,Secretaría de Vivienda Social y Habitat,2025-11-18T00:50:28.595762,UNP-3,POINT (3.4004 -76.52156),La Alborada,COMUNA 16,ACEPTABLE,2025-06-24,2025-11-30
3,4244.0.9.10.341-2025,None,2024760010165,Subsidios para Cierre Financiero,Vivienda Nueva,Empréstito,Asignación del Subsidio Distrital de Vivienda ...,Asignación del Subsidio Distrital de Vivienda ...,EXPANSION URBANA,Obra nueva,...,None,Secretaría de Vivienda Social y Habitat,2025-11-18T00:50:28.595762,UNP-4,POINT (3.34194 -76.51078),NaN,NaN,FUERA DE RANGO,2025-06-24,2025-11-30
4,4244.0.9.10.341-2025,None,2024760010165,Subsidios para Cierre Financiero,Vivienda Nueva,Empréstito,Asignación del Subsidio Distrital de Vivienda ...,Asignación del Subsidio Distrital de Vivienda ...,COMUNA 16,Obra nueva,...,None,Secretaría de Vivienda Social y Habitat,2025-11-18T00:50:28.595762,UNP-5,POINT (3.4004 -76.52156),La Alborada,COMUNA 16,ACEPTABLE,2025-06-24,2025-11-30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1636,4164.010.32.1.571-2024,4164.010.26.1.578-2024,2024760010015,Casa de las Memorias,Infraestructura Cultural,Recursos propios,Casa de las Memorias,Casa de las Memorias,COMUNA 03,Obra nueva,...,None,Secretaría de Paz y Cultura Ciudadana,2025-11-18T00:50:28.595762,UNP-1637,POINT (3.45035 -76.53638),La Merced,COMUNA 03,ACEPTABLE,2024-11-12,2024-12-12
1637,OC143697,OC143697,2024760010008,Demarcacion_plastico_frio,Señalización Vial,Ingresos con destinación específica,Demarcacion_plastico_frio,Adquirir Insumos para la Demarcacion de Vias d...,COMUNAS DE CALI,Obra nueva,...,None,Secretaría de Movilidad,2025-11-18T00:50:28.595762,UNP-1638,None,NaN,NaN,None,2025-04-02,2025-07-20
1638,OC149988,OC149988,2024760010107,Demarcacion_cicloinfraestrutura,Señalización Vial,Ingresos con destinación específica,Demarcacion_cicloinfraestrutura,Suministro de Insumos para la Demarcacion de C...,COMUNAS DE CALI,Obra nueva,...,None,Secretaría de Movilidad,2025-11-18T00:50:28.595762,UNP-1639,None,NaN,NaN,None,2025-08-26,2025-10-31
1639,4152.010.26.1.598-2025,4152.010.26.1.598-2025,2024760010008,Reductores,Señalización Vial,Ingresos con destinación específica,Reductores,"Construccion, Instalacion, Demolicion, y Repos...",COMUNAS DE CALI,Obra nueva,...,None,Secretaría de Movilidad,2025-11-18T00:50:28.595762,UNP-1640,None,NaN,NaN,None,2025-07-14,2025-12-31


In [37]:
gdf['comuna_corregimiento'].unique()

array(['COMUNA 18', 'EXPANSION URBANA', 'COMUNA 16', 'COMUNA 08',
       'COMUNA 19', 'COMUNA 13', 'DISTRITO', 'COMUNA 14', 'COMUNA 03',
       'COMUNA 21', 'COMUNA 10', 'Los Andes', 'COMUNA 09', 'COMUNA 01',
       'COMUNA 20', 'COMUNA 15', 'COMUNA 07', 'Distrito', 'COMUNA 12',
       'COMUNA 22', 'COMUNA 17', 'COMUNA 04', 'COMUNA 02', 'COMUNA 06',
       'Golondrinas', 'COMUNA 11', 'El Saladito', 'COMUNA 05',
       'El Hormiguero', 'EXPANSIÓN', 'Felidia', 'La Buitrera',
       'La Castilla', 'La Elvira', 'La Leonera', 'Montebello', 'Navarro',
       'Villacarmelo', 'Pance', 'RURAL', 'Pichinde', 'Corregimiento',
       'COMUNAS DE CALI', 'La Paz'], dtype=object)

In [38]:
gdf['barrio_vereda'].unique()

array(['Polvorines', 'Plan Parcial El Capricho', 'La Alborada',
       'Primitivo Crespo', 'Bellavista', 'Ricardo Balcázar',
       'Plan Parcial Guayabal', 'Industrial',
       'Plan Parcial Ciudad Melendez', 'Plan Parcial Cachipay',
       'Distrito', nan, 'Corregimiento de los Andes  ',
       'Corregimiento de los Andes  \n\nQuebrada el venado ', 'Calipso',
       'Charco Azul y El Pondaje ', 'San Pascual', 'El Calvario',
       'Pizamos I', 'San Fernando Viejo', 'Camino Real', 'Pasoancho',
       'Los Cómbulos', 'El Refugio', 'Vereda El Mameyal\n',
       'Vereda El Mameyal', 'Sucre', 'Barrio Obrero', 'Vista Hermosa',
       'Sector Patio Bonito', 'Tierra Blanca', 'Siloé', 'Los Robles',
       'El Poblado I', 'El Poblado II', 'Charco Azul', 'El Pondaje',
       'El Vergel', 'Los Lagos', 'Manuela Beltrán',
       'José Manuel Marroquín II', 'Promociones Populares B',
       'Puerta del Sol', 'Alfonso Bonilla Aragón',
       'José Manuel Marroquín I', 'Las Orquídeas', 'Mojica',
    

In [39]:
gdf['comuna_corregimiento_2'].unique()

array(['COMUNA 09', nan, 'COMUNA 16', 'COMUNA 08', 'COMUNA 19',
       'COMUNA 13', 'COMUNA 15', 'Los Andes', 'COMUNA 03', 'COMUNA 21',
       'COMUNA 10', 'COMUNA 01', 'COMUNA 20', 'COMUNA 14', 'COMUNA 18',
       'COMUNA 07', 'COMUNA 12', 'COMUNA 22', 'COMUNA 17', 'COMUNA 04',
       'Golondrinas', 'COMUNA 06', 'COMUNA 02', 'COMUNA 11',
       'El Saladito', 'COMUNA 05', 'La Buitrera', 'Navarro', 'Pichinde',
       'La Leonera', 'El Hormiguero', 'Pance', 'Montebello', 'La Paz'],
      dtype=object)

In [40]:
gdf['barrio_vereda_2'].unique()

array(['Belalcázar', nan, 'La Alborada', 'Primitivo Crespo', 'Bellavista',
       'Sector Laguna del Pondaje', 'Industrial', 'Mojica',
       'Ecoparque Cristo Rey', 'Pilas del Cabuyal', 'Calipso',
       'San Pascual', 'El Calvario', 'Pizamos I', 'San Fernando Viejo',
       'Camino Real - Joaquin Borrero Sinisterra', 'Pasoancho',
       'Los Cómbulos', 'El Refugio', 'Mameyal', 'Brisas de los Cristales',
       'Sector Altos de Santa Isabel', 'Sucre', 'Barrio Obrero',
       'Sector Patio Bonito', 'Siloé', 'Los Robles', 'El Poblado I',
       'El Poblado II', 'Charco Azul', 'El Pondaje', 'El Vergel',
       'Los Lagos', 'Manuela Beltrán', 'José Manuel Marroquín II',
       'Promociones Populares B', 'José Manuel Marroquín I',
       'Alfonso Bonilla Aragón', 'Las Orquídeas', 'Puerta del Sol',
       'Los Comuneros I', 'Ciudad Córdoba', 'Marroquín III',
       'Villablanca', 'Omar Torrijos', 'Rodrigo Lara Bonilla',
       'Alirio Mora Beltrán', 'El Vallado', 'Laureano Gómez', 'El Retir

In [41]:
gdf['estado'].unique()

array(['En alistamiento', 'En ejecución', 'En liquidación', 'Finalizado'],
      dtype=object)

In [42]:
gdf['tipo_intervencion'].unique()

array(['Obra nueva', nan, 'Rehabilitación - Reforzamiento',
       'Adecuaciones y mantenimientos', 'Adecuaciones y Mantenimientos'],
      dtype=object)

## 15. Exportar Datos (Opcional)

Guardar los datos transformados en formato JSON o CSV si es necesario.

### 15.1. CREACIÓN DE GEOJSON PARA CARGA

In [52]:
import json
from pathlib import Path

print("=" * 60)
print("EXPORTANDO GEODATAFRAME A GEOJSON")
print("=" * 60)
print()

if gdf is not None:
    # Define output path
    output_dir = workspace_path / 'app_outputs'
    output_dir.mkdir(exist_ok=True)
    
    output_file = output_dir / 'unidades_proyecto_transformed.geojson'
    
    # Create a copy for export
    gdf_export = gdf.copy()
    
    # Convert geometry back to standard lon, lat format for GeoJSON export
    # Current format is Point(lat, lon), need to convert to Point(lon, lat)
    valid_geom = gdf_export['geometry'].notna()
    
    if valid_geom.sum() > 0:
        print("Convirtiendo geometrías a formato GeoJSON estándar (lon, lat)...")
        gdf_export.loc[valid_geom, 'geometry'] = gdf_export.loc[valid_geom, 'geometry'].apply(
            lambda geom: Point(geom.y, geom.x) if geom else None  # Swap: x=lat, y=lon -> Point(lon, lat)
        )
    
    # Convert datetime columns to string for JSON serialization
    date_columns = ['fecha_inicio_std', 'fecha_fin_std']
    for col in date_columns:
        if col in gdf_export.columns:
            gdf_export[col] = gdf_export[col].apply(
                lambda x: x.isoformat() if pd.notna(x) and hasattr(x, 'isoformat') else None
            )
    
    # Export to GeoJSON
    gdf_export.to_file(output_file, driver='GeoJSON')
    
    print(f"✓ GeoDataFrame exportado exitosamente")
    print(f"  Archivo: {output_file}")
    print(f"  Tamaño: {output_file.stat().st_size / 1024:.2f} KB")
    print()
    
    # Display export statistics
    print("ESTADÍSTICAS DEL ARCHIVO EXPORTADO:")
    print("-" * 60)
    print(f"Total de registros: {len(gdf_export)}")
    print(f"Registros con geometría: {gdf_export['geometry'].notna().sum()}")
    print(f"Registros sin geometría: {gdf_export['geometry'].isna().sum()}")
    print(f"Total de columnas: {len(gdf_export.columns)}")
    print(f"Formato de geometría: Point(lon, lat) - Estándar GeoJSON")
    print(f"CRS: {gdf_export.crs}")
    print()
    
    # Show column list
    print("Columnas exportadas:")
    for i, col in enumerate(gdf_export.columns, 1):
        print(f"  {i}. {col}")
    
    print()
    print("=" * 60)
    print("✓ EXPORTACIÓN COMPLETADA")
    print("=" * 60)
    
else:
    print("⚠ Error: GeoDataFrame no disponible para exportar")

EXPORTANDO GEODATAFRAME A GEOJSON

Convirtiendo geometrías a formato GeoJSON estándar (lon, lat)...
✓ GeoDataFrame exportado exitosamente
  Archivo: a:\programing_workspace\proyectos_cali_alcaldia_etl\app_outputs\unidades_proyecto_transformed.geojson
  Tamaño: 2100.21 KB

ESTADÍSTICAS DEL ARCHIVO EXPORTADO:
------------------------------------------------------------
Total de registros: 1641
Registros con geometría: 1561
Registros sin geometría: 80
Total de columnas: 32
Formato de geometría: Point(lon, lat) - Estándar GeoJSON
CRS: EPSG:4326

Columnas exportadas:
  1. referencia_proceso
  2. referencia_contrato
  3. bpin
  4. identificador
  5. tipo_equipamiento
  6. fuente_financiacion
  7. nombre_up
  8. nombre_up_detalle
  9. comuna_corregimiento
  10. tipo_intervencion
  11. unidad
  12. cantidad
  13. direccion
  14. barrio_vereda
  15. estado
  16. presupuesto_base
  17. avance_obra
  18. ano
  19. fecha_inicio
  20. fecha_fin
  21. plataforma
  22. url_proceso
  23. descripcion_i

### 15.2. CREACIÓN DE REPORTE - CALIDAD DE DATOS EN UNIDADES DE PROYECTO

In [54]:
import json
from datetime import datetime
from pathlib import Path

print("=" * 60)
print("GENERANDO LOG DE MÉTRICAS DEL PROCESO DE TRANSFORMACIÓN")
print("=" * 60)
print()

# Initialize metrics dictionary
metrics = {
    "execution_timestamp": datetime.now().isoformat(),
    "process_name": "Transformación de Unidades de Proyecto",
    "version": "3.0",
    
    # Data loading metrics
    "data_loading": {
        "total_records_loaded": len(df),
        "total_columns_loaded": len(df.columns),
        "timestamp": datetime.now().isoformat()
    },
    
    # Data transformation metrics
    "data_transformation": {
        "total_records_transformed": len(gdf),
        "total_columns_final": len(gdf.columns),
        "upid_generated": (gdf['upid'].notna()).sum(),
        "timestamp": datetime.now().isoformat()
    },
    
    # Data type cleaning metrics
    "data_cleaning": {
        "text_columns_cleaned": len(['nickname_detalle', 'direccion', 'descripcion_intervencion', 'identificador', 'nickname']),
        "monetary_columns_cleaned": len(['presupuesto_base']),
        "numeric_columns_cleaned": len(['avance_obra', 'avance_fisico_obra']),
        "timestamp": datetime.now().isoformat()
    },
    
    # Reference processing metrics
    "reference_processing": {
        "referencia_proceso_single": (gdf['referencia_proceso'].apply(lambda x: isinstance(x, str))).sum(),
        "referencia_proceso_list": (gdf['referencia_proceso'].apply(lambda x: isinstance(x, list))).sum(),
        "referencia_contrato_single": (gdf['referencia_contrato'].apply(lambda x: isinstance(x, str))).sum(),
        "referencia_contrato_list": (gdf['referencia_contrato'].apply(lambda x: isinstance(x, list))).sum(),
        "timestamp": datetime.now().isoformat()
    },
    
    # Monetary validation metrics
    "monetary_validation": {
        "presupuesto_base_positive": (gdf['presupuesto_base'] > 0).sum(),
        "presupuesto_base_zero": (gdf['presupuesto_base'] == 0).sum(),
        "presupuesto_base_total": gdf['presupuesto_base'].sum(),
        "timestamp": datetime.now().isoformat()
    },
    
    # Geospatial metrics
    "geospatial_processing": {
        "total_records": len(gdf),
        "records_with_geometry": (gdf['geometry'].notna()).sum(),
        "records_without_geometry": (gdf['geometry'].isna()).sum(),
        "records_within_cali_bounds": len(gdf[gdf['geometry'].notna()]) - len(records_outside) if 'records_outside' in locals() else 0,
        "records_outside_cali_bounds": len(records_outside) if 'records_outside' in locals() else 0,
        "crs": str(gdf.crs),
        "timestamp": datetime.now().isoformat()
    },
    
    # Spatial intersection metrics
    "spatial_intersection": {
        "barrio_vereda_2_assigned": (gdf['barrio_vereda_2'].notna()).sum(),
        "barrio_vereda_2_null": (gdf['barrio_vereda_2'].isna()).sum(),
        "comuna_corregimiento_2_assigned": (gdf['comuna_corregimiento_2'].notna()).sum(),
        "comuna_corregimiento_2_null": (gdf['comuna_corregimiento_2'].isna()).sum(),
        "timestamp": datetime.now().isoformat()
    },
    
    # Data validation metrics
    "validation": {
        "fuera_rango_aceptable": (gdf['fuera_rango'] == 'ACEPTABLE').sum(),
        "fuera_rango_invalid": (gdf['fuera_rango'] == 'FUERA DE RANGO').sum(),
        "fuera_rango_null": (gdf['fuera_rango'].isna()).sum(),
        "timestamp": datetime.now().isoformat()
    },
    
    # Date standardization metrics
    "date_processing": {
        "fecha_inicio_valid": (gdf['fecha_inicio_std'].notna()).sum() if 'fecha_inicio_std' in gdf.columns else 0,
        "fecha_inicio_null": (gdf['fecha_inicio_std'].isna()).sum() if 'fecha_inicio_std' in gdf.columns else len(gdf),
        "fecha_fin_valid": (gdf['fecha_fin_std'].notna()).sum() if 'fecha_fin_std' in gdf.columns else 0,
        "fecha_fin_null": (gdf['fecha_fin_std'].isna()).sum() if 'fecha_fin_std' in gdf.columns else len(gdf),
        "invalid_date_order": len(invalid_order) if 'invalid_order' in locals() else 0,
        "timestamp": datetime.now().isoformat()
    },
    
    # Normalization metrics
    "normalization": {
        "comuna_values_normalized": len(changes_comuna) if 'changes_comuna' in locals() else 0,
        "barrio_values_normalized": len(changes_barrio) if 'changes_barrio' in locals() else 0,
        "timestamp": datetime.now().isoformat()
    },
    
    # Column type distribution
    "column_types": {
        "text_columns": len(gdf.select_dtypes(include=['object']).columns),
        "numeric_columns": len(gdf.select_dtypes(include=['number']).columns),
        "boolean_columns": len(gdf.select_dtypes(include=['bool']).columns),
        "geometry_columns": 1,
        "timestamp": datetime.now().isoformat()
    },
    
    # Null values summary
    "null_values": {
        "columns_with_nulls": len(gdf.isnull().sum()[gdf.isnull().sum() > 0]),
        "total_null_values": gdf.isnull().sum().sum(),
        "timestamp": datetime.now().isoformat()
    },
    
    # Estado distribution
    "estado_distribution": gdf['estado'].value_counts().to_dict() if 'estado' in gdf.columns else {},
    
    # Tipo intervención distribution
    "tipo_intervencion_distribution": gdf['tipo_intervencion'].value_counts().to_dict() if 'tipo_intervencion' in gdf.columns else {},
    
    # Export information
    "export": {
        "file_exported": str(output_file) if 'output_file' in locals() else None,
        "file_size_kb": output_file.stat().st_size / 1024 if 'output_file' in locals() and output_file.exists() else 0,
        "export_format": "GeoJSON",
        "timestamp": datetime.now().isoformat()
    },
    
    # Process summary
    "summary": {
        "total_execution_time": "Calculated separately",
        "data_quality_score": round((gdf['fuera_rango'] == 'ACEPTABLE').sum() / len(gdf) * 100, 2),
        "geometry_completeness": round((gdf['geometry'].notna()).sum() / len(gdf) * 100, 2),
        "date_completeness": round((gdf['fecha_inicio_std'].notna()).sum() / len(gdf) * 100, 2) if 'fecha_inicio_std' in gdf.columns else 0,
        "timestamp": datetime.now().isoformat()
    }
}

# Convert numpy types to native Python types for JSON serialization
def convert_to_native_types(obj):
    """Recursively convert numpy types to native Python types."""
    import numpy as np
    
    if isinstance(obj, dict):
        return {key: convert_to_native_types(value) for key, value in obj.items()}
    elif isinstance(obj, list):
        return [convert_to_native_types(item) for item in obj]
    elif isinstance(obj, (np.integer, np.int64, np.int32)):
        return int(obj)
    elif isinstance(obj, (np.floating, np.float64, np.float32)):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    else:
        return obj

# Convert metrics to native Python types
metrics_serializable = convert_to_native_types(metrics)

# Save metrics to JSON file
metrics_output_dir = workspace_path / 'app_outputs' / 'logs'
metrics_output_dir.mkdir(exist_ok=True, parents=True)

timestamp_str = datetime.now().strftime("%Y%m%d_%H%M%S")
metrics_file = metrics_output_dir / f'transformation_metrics_{timestamp_str}.json'

with open(metrics_file, 'w', encoding='utf-8') as f:
    json.dump(metrics_serializable, f, ensure_ascii=False, indent=2)

print(f"✓ Métricas guardadas exitosamente")
print(f"  Archivo: {metrics_file}")
print(f"  Tamaño: {metrics_file.stat().st_size / 1024:.2f} KB")
print()

# Display summary metrics
print("RESUMEN DE MÉTRICAS:")
print("-" * 60)
print(f"Timestamp de ejecución: {metrics['execution_timestamp']}")
print(f"Total de registros: {metrics['data_transformation']['total_records_transformed']}")
print(f"Registros con geometría: {metrics['geospatial_processing']['records_with_geometry']}")
print(f"Calidad de datos: {metrics['summary']['data_quality_score']}%")
print(f"Completitud de geometría: {metrics['summary']['geometry_completeness']}%")
print(f"Completitud de fechas: {metrics['summary']['date_completeness']}%")
print()

# Display top 5 estados
if metrics['estado_distribution']:
    print("Top 5 Estados:")
    for estado, count in sorted(metrics['estado_distribution'].items(), key=lambda x: x[1], reverse=True)[:5]:
        print(f"  {estado}: {count}")
    print()

# Display top 5 tipos de intervención
if metrics['tipo_intervencion_distribution']:
    print("Top 5 Tipos de Intervención:")
    for tipo, count in sorted(metrics['tipo_intervencion_distribution'].items(), key=lambda x: x[1], reverse=True)[:5]:
        print(f"  {tipo}: {count}")
    print()

print("=" * 60)
print("✓ LOG DE MÉTRICAS COMPLETADO")
print("=" * 60)

GENERANDO LOG DE MÉTRICAS DEL PROCESO DE TRANSFORMACIÓN

✓ Métricas guardadas exitosamente
  Archivo: a:\programing_workspace\proyectos_cali_alcaldia_etl\app_outputs\logs\transformation_metrics_20251116_023227.json
  Tamaño: 3.23 KB

RESUMEN DE MÉTRICAS:
------------------------------------------------------------
Timestamp de ejecución: 2025-11-16T02:32:27.738643
Total de registros: 1641
Registros con geometría: 1561
Calidad de datos: 67.7%
Completitud de geometría: 95.12%
Completitud de fechas: 84.4%

Top 5 Estados:
  En alistamiento: 1125
  En ejecución: 287
  Finalizado: 226
  En liquidación: 3

Top 5 Tipos de Intervención:
  Obra nueva: 942
  Adecuaciones y mantenimientos: 628
  Adecuaciones y Mantenimientos: 56
  Rehabilitación - Reforzamiento: 14

✓ LOG DE MÉTRICAS COMPLETADO


In [55]:
import json
from datetime import datetime
from pathlib import Path

print("=" * 80)
print("GENERANDO REPORTE DE ANÁLISIS Y RECOMENDACIONES")
print("=" * 80)
print()

# Load the most recent metrics file
if 'metrics_file' in locals() and metrics_file.exists():
    with open(metrics_file, 'r', encoding='utf-8') as f:
        metrics_data = json.load(f)
    
    print(f"✓ Métricas cargadas desde: {metrics_file.name}")
    print()
else:
    print("⚠ No se encontraron métricas previas")
    metrics_data = {}

# Calculate additional analysis metrics
total_records = metrics_data.get('data_transformation', {}).get('total_records_transformed', 0)
quality_score = metrics_data.get('summary', {}).get('data_quality_score', 0)
geometry_completeness = metrics_data.get('summary', {}).get('geometry_completeness', 0)
date_completeness = metrics_data.get('summary', {}).get('date_completeness', 0)

# Geospatial metrics
records_with_geometry = metrics_data.get('geospatial_processing', {}).get('records_with_geometry', 0)
records_outside_bounds = metrics_data.get('geospatial_processing', {}).get('records_outside_cali_bounds', 0)

# Validation metrics
acceptable_records = metrics_data.get('validation', {}).get('fuera_rango_aceptable', 0)
invalid_records = metrics_data.get('validation', {}).get('fuera_rango_invalid', 0)

# Date processing metrics
date_valid_inicio = metrics_data.get('date_processing', {}).get('fecha_inicio_valid', 0)
date_invalid_order = metrics_data.get('date_processing', {}).get('invalid_date_order', 0)

# Normalization metrics
comuna_normalized = metrics_data.get('normalization', {}).get('comuna_values_normalized', 0)
barrio_normalized = metrics_data.get('normalization', {}).get('barrio_values_normalized', 0)

# Reference processing metrics
ref_proceso_list = metrics_data.get('reference_processing', {}).get('referencia_proceso_list', 0)
ref_contrato_list = metrics_data.get('reference_processing', {}).get('referencia_contrato_list', 0)

# Calculate quality indicators
geometry_quality = "EXCELENTE" if geometry_completeness >= 95 else "BUENA" if geometry_completeness >= 85 else "REGULAR" if geometry_completeness >= 70 else "DEFICIENTE"
spatial_quality = "EXCELENTE" if quality_score >= 90 else "BUENA" if quality_score >= 75 else "REGULAR" if quality_score >= 60 else "DEFICIENTE"
date_quality = "EXCELENTE" if date_completeness >= 95 else "BUENA" if date_completeness >= 80 else "REGULAR" if date_completeness >= 60 else "DEFICIENTE"

# Generate recommendations based on metrics
recommendations = []

# Geometry recommendations
if geometry_completeness < 95:
    missing_geom = total_records - records_with_geometry
    recommendations.append({
        "categoria": "Datos Geoespaciales",
        "prioridad": "ALTA",
        "issue": f"{missing_geom} registros ({100-geometry_completeness:.1f}%) sin coordenadas geográficas",
        "impacto": "Limita la capacidad de análisis espacial y visualización en mapas",
        "recomendacion": "Implementar proceso de geocodificación para registros sin coordenadas usando direcciones disponibles"
    })

# Spatial validation recommendations
if invalid_records > 0:
    invalid_percentage = (invalid_records / total_records) * 100
    recommendations.append({
        "categoria": "Validación Espacial",
        "prioridad": "ALTA" if invalid_percentage > 20 else "MEDIA",
        "issue": f"{invalid_records} registros ({invalid_percentage:.1f}%) con inconsistencias entre ubicación y datos administrativos",
        "impacto": "Coordenadas no coinciden con comuna/barrio declarado, indica posibles errores de georreferenciación",
        "recomendacion": "Revisar y corregir coordenadas de registros FUERA DE RANGO mediante validación manual o re-geocodificación"
    })

# Out of bounds recommendations
if records_outside_bounds > 0:
    recommendations.append({
        "categoria": "Límites Geográficos",
        "prioridad": "ALTA",
        "issue": f"{records_outside_bounds} registros con coordenadas fuera de los límites de Santiago de Cali",
        "impacto": "Coordenadas incorrectas que no corresponden a la ciudad",
        "recomendacion": "Verificar y corregir coordenadas de estos registros, posiblemente intercambio de lat/lon o datos erróneos"
    })

# Date recommendations
if date_completeness < 85:
    missing_dates = total_records - date_valid_inicio
    recommendations.append({
        "categoria": "Datos Temporales",
        "prioridad": "MEDIA",
        "issue": f"{missing_dates} registros ({100-date_completeness:.1f}%) sin fecha de inicio",
        "impacto": "Dificulta análisis temporal y seguimiento de cronogramas",
        "recomendacion": "Completar fechas faltantes consultando fuentes primarias (SECOP, documentos contractuales)"
    })

# Date order recommendations
if date_invalid_order > 0:
    recommendations.append({
        "categoria": "Consistencia Temporal",
        "prioridad": "MEDIA",
        "issue": f"{date_invalid_order} registros con fecha_fin anterior a fecha_inicio",
        "impacto": "Inconsistencia lógica que invalida cálculos de duración de proyectos",
        "recomendacion": "Revisar y corregir el orden de fechas, posiblemente intercambio o errores de captura"
    })

# Normalization recommendations
total_normalized = comuna_normalized + barrio_normalized
if total_normalized > 0:
    recommendations.append({
        "categoria": "Normalización de Datos",
        "prioridad": "BAJA",
        "issue": f"{total_normalized} valores normalizados ({comuna_normalized} comunas, {barrio_normalized} barrios)",
        "impacto": "Inconsistencias menores en nomenclatura que afectan agregaciones",
        "recomendacion": "Implementar validación en origen para asegurar uso de catálogos estandarizados"
    })

# Reference list recommendations
total_list_refs = ref_proceso_list + ref_contrato_list
if total_list_refs > 0:
    recommendations.append({
        "categoria": "Referencias Múltiples",
        "prioridad": "BAJA",
        "issue": f"{total_list_refs} proyectos con múltiples referencias ({ref_proceso_list} procesos, {ref_contrato_list} contratos)",
        "impacto": "Complejidad en trazabilidad, pero manejado correctamente",
        "recomendacion": "Considerar crear tabla relacional para manejar relaciones uno-a-muchos de forma normalizada"
    })

# Monetary validation
presupuesto_cero = metrics_data.get('monetary_validation', {}).get('presupuesto_base_zero', 0)
if presupuesto_cero > 0:
    recommendations.append({
        "categoria": "Datos Presupuestales",
        "prioridad": "MEDIA",
        "issue": f"{presupuesto_cero} registros con presupuesto_base en $0",
        "impacto": "Impide análisis de inversión y priorización por monto",
        "recomendacion": "Completar información presupuestal desde fuentes oficiales (SECOP, POA institucional)"
    })

# Create comprehensive report
report = {
    "metadata": {
        "titulo": "Reporte de Análisis y Recomendaciones - Transformación de Unidades de Proyecto",
        "version": "1.0",
        "fecha_generacion": datetime.now().isoformat(),
        "fecha_ejecucion_etl": metrics_data.get('execution_timestamp'),
        "archivo_metricas": str(metrics_file.name) if 'metrics_file' in locals() else None
    },
    
    "resumen_ejecutivo": {
        "total_registros": total_records,
        "calidad_global": {
            "score": quality_score,
            "nivel": spatial_quality,
            "interpretacion": f"{'Excelente calidad' if quality_score >= 90 else 'Buena calidad' if quality_score >= 75 else 'Calidad aceptable' if quality_score >= 60 else 'Requiere mejoras significativas'}"
        },
        "indicadores_clave": {
            "completitud_geometrica": {
                "porcentaje": geometry_completeness,
                "nivel": geometry_quality,
                "registros_con_geometria": records_with_geometry,
                "registros_sin_geometria": total_records - records_with_geometry
            },
            "completitud_temporal": {
                "porcentaje": date_completeness,
                "nivel": date_quality,
                "registros_con_fechas": date_valid_inicio,
                "registros_sin_fechas": total_records - date_valid_inicio
            },
            "validacion_espacial": {
                "registros_aceptables": acceptable_records,
                "registros_invalidos": invalid_records,
                "porcentaje_aceptable": quality_score,
                "registros_fuera_limites": records_outside_bounds
            }
        }
    },
    
    "analisis_detallado": {
        "procesamiento_datos": {
            "registros_cargados": metrics_data.get('data_loading', {}).get('total_records_loaded', 0),
            "registros_transformados": total_records,
            "columnas_finales": metrics_data.get('data_transformation', {}).get('total_columns_final', 0),
            "upid_generados": metrics_data.get('data_transformation', {}).get('upid_generated', 0)
        },
        
        "limpieza_datos": {
            "columnas_texto_limpiadas": metrics_data.get('data_cleaning', {}).get('text_columns_cleaned', 0),
            "columnas_monetarias_limpiadas": metrics_data.get('data_cleaning', {}).get('monetary_columns_cleaned', 0),
            "columnas_numericas_limpiadas": metrics_data.get('data_cleaning', {}).get('numeric_columns_cleaned', 0)
        },
        
        "procesamiento_referencias": {
            "referencias_proceso_simples": metrics_data.get('reference_processing', {}).get('referencia_proceso_single', 0),
            "referencias_proceso_multiples": ref_proceso_list,
            "referencias_contrato_simples": metrics_data.get('reference_processing', {}).get('referencia_contrato_single', 0),
            "referencias_contrato_multiples": ref_contrato_list
        },
        
        "validacion_presupuestal": {
            "registros_con_presupuesto": metrics_data.get('monetary_validation', {}).get('presupuesto_base_positive', 0),
            "registros_sin_presupuesto": presupuesto_cero,
            "presupuesto_total": f"${metrics_data.get('monetary_validation', {}).get('presupuesto_base_total', 0):,.0f}"
        },
        
        "procesamiento_geoespacial": {
            "registros_geocodificados": records_with_geometry,
            "registros_sin_geocodificar": total_records - records_with_geometry,
            "registros_dentro_limites": metrics_data.get('geospatial_processing', {}).get('records_within_cali_bounds', 0),
            "registros_fuera_limites": records_outside_bounds,
            "sistema_coordenadas": metrics_data.get('geospatial_processing', {}).get('crs')
        },
        
        "interseccion_espacial": {
            "barrios_asignados": metrics_data.get('spatial_intersection', {}).get('barrio_vereda_2_assigned', 0),
            "barrios_sin_asignar": metrics_data.get('spatial_intersection', {}).get('barrio_vereda_2_null', 0),
            "comunas_asignadas": metrics_data.get('spatial_intersection', {}).get('comuna_corregimiento_2_assigned', 0),
            "comunas_sin_asignar": metrics_data.get('spatial_intersection', {}).get('comuna_corregimiento_2_null', 0)
        },
        
        "normalizacion": {
            "valores_comuna_normalizados": comuna_normalized,
            "valores_barrio_normalizados": barrio_normalized,
            "total_normalizaciones": total_normalized
        },
        
        "procesamiento_fechas": {
            "fechas_inicio_validas": date_valid_inicio,
            "fechas_inicio_invalidas": total_records - date_valid_inicio,
            "fechas_fin_validas": metrics_data.get('date_processing', {}).get('fecha_fin_valid', 0),
            "fechas_con_orden_invalido": date_invalid_order
        }
    },
    
    "distribucion_datos": {
        "por_estado": metrics_data.get('estado_distribution', {}),
        "por_tipo_intervencion": metrics_data.get('tipo_intervencion_distribution', {})
    },
    
    "recomendaciones": recommendations,
    
    "acciones_prioritarias": [
        {
            "prioridad": 1,
            "accion": "Corregir coordenadas de registros fuera de límites de Cali",
            "registros_afectados": records_outside_bounds,
            "impacto_esperado": "Alto - Mejora significativa en validación espacial"
        } if records_outside_bounds > 0 else None,
        {
            "prioridad": 2,
            "accion": "Revisar y corregir registros con validación espacial FUERA DE RANGO",
            "registros_afectados": invalid_records,
            "impacto_esperado": f"Alto - Incrementaría calidad espacial de {quality_score:.1f}% a ~{min(100, quality_score + (invalid_records/total_records)*100):.1f}%"
        } if invalid_records > 0 else None,
        {
            "prioridad": 3,
            "accion": "Geocodificar registros sin coordenadas",
            "registros_afectados": total_records - records_with_geometry,
            "impacto_esperado": f"Medio - Incrementaría completitud geométrica de {geometry_completeness:.1f}% a 100%"
        } if geometry_completeness < 100 else None,
        {
            "prioridad": 4,
            "accion": "Completar fechas faltantes",
            "registros_afectados": total_records - date_valid_inicio,
            "impacto_esperado": f"Medio - Incrementaría completitud temporal de {date_completeness:.1f}% a ~100%"
        } if date_completeness < 100 else None,
        {
            "prioridad": 5,
            "accion": "Corregir orden de fechas (fecha_fin < fecha_inicio)",
            "registros_afectados": date_invalid_order,
            "impacto_esperado": "Bajo - Mejora consistencia temporal"
        } if date_invalid_order > 0 else None
    ],
    
    "metricas_calidad": {
        "completitud": {
            "geometrica": geometry_completeness,
            "temporal": date_completeness,
            "presupuestal": (metrics_data.get('monetary_validation', {}).get('presupuesto_base_positive', 0) / total_records * 100) if total_records > 0 else 0
        },
        "consistencia": {
            "espacial": quality_score,
            "temporal": ((total_records - date_invalid_order) / total_records * 100) if total_records > 0 else 0,
            "referencial": ((total_records - total_list_refs) / total_records * 100) if total_records > 0 else 0
        },
        "precision": {
            "dentro_limites_geograficos": ((records_with_geometry - records_outside_bounds) / records_with_geometry * 100) if records_with_geometry > 0 else 0,
            "validacion_administrativa": quality_score,
            "normalizacion_nomenclatura": ((total_records - total_normalized) / total_records * 100) if total_records > 0 else 0
        }
    },
    
    "exportacion": {
        "archivo_generado": metrics_data.get('export', {}).get('file_exported'),
        "formato": metrics_data.get('export', {}).get('export_format'),
        "tamano_kb": metrics_data.get('export', {}).get('file_size_kb'),
        "timestamp": metrics_data.get('export', {}).get('timestamp')
    }
}

# Filter out None values from acciones_prioritarias
report["acciones_prioritarias"] = [action for action in report["acciones_prioritarias"] if action is not None]

# Save report as JSON
report_output_dir = workspace_path / 'app_outputs' / 'reports'
report_output_dir.mkdir(exist_ok=True, parents=True)

timestamp_str = datetime.now().strftime("%Y%m%d_%H%M%S")
report_json_file = report_output_dir / f'analisis_recomendaciones_{timestamp_str}.json'

with open(report_json_file, 'w', encoding='utf-8') as f:
    json.dump(report, f, ensure_ascii=False, indent=2)

print(f"✓ Reporte JSON guardado: {report_json_file.name}")
print(f"  Tamaño: {report_json_file.stat().st_size / 1024:.2f} KB")
print()

# Generate Markdown report
md_lines = [
    "# Reporte de Análisis y Recomendaciones",
    "## Transformación de Unidades de Proyecto",
    "",
    f"**Fecha de Generación:** {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}  ",
    f"**Versión:** {report['metadata']['version']}  ",
    f"**Archivo de Métricas:** `{report['metadata']['archivo_metricas']}`",
    "",
    "---",
    "",
    "## 📊 Resumen Ejecutivo",
    "",
    f"**Total de Registros Procesados:** {report['resumen_ejecutivo']['total_registros']:,}",
    "",
    "### Calidad Global",
    f"- **Score de Calidad:** {report['resumen_ejecutivo']['calidad_global']['score']:.1f}% ({report['resumen_ejecutivo']['calidad_global']['nivel']})",
    f"- **Interpretación:** {report['resumen_ejecutivo']['calidad_global']['interpretacion']}",
    "",
    "### Indicadores Clave",
    "",
    "#### 🗺️ Completitud Geométrica",
    f"- **Nivel:** {report['resumen_ejecutivo']['indicadores_clave']['completitud_geometrica']['nivel']}",
    f"- **Porcentaje:** {report['resumen_ejecutivo']['indicadores_clave']['completitud_geometrica']['porcentaje']:.1f}%",
    f"- **Con Geometría:** {report['resumen_ejecutivo']['indicadores_clave']['completitud_geometrica']['registros_con_geometria']:,} registros",
    f"- **Sin Geometría:** {report['resumen_ejecutivo']['indicadores_clave']['completitud_geometrica']['registros_sin_geometria']:,} registros",
    "",
    "#### 📅 Completitud Temporal",
    f"- **Nivel:** {report['resumen_ejecutivo']['indicadores_clave']['completitud_temporal']['nivel']}",
    f"- **Porcentaje:** {report['resumen_ejecutivo']['indicadores_clave']['completitud_temporal']['porcentaje']:.1f}%",
    f"- **Con Fechas:** {report['resumen_ejecutivo']['indicadores_clave']['completitud_temporal']['registros_con_fechas']:,} registros",
    f"- **Sin Fechas:** {report['resumen_ejecutivo']['indicadores_clave']['completitud_temporal']['registros_sin_fechas']:,} registros",
    "",
    "#### ✅ Validación Espacial",
    f"- **Registros Aceptables:** {report['resumen_ejecutivo']['indicadores_clave']['validacion_espacial']['registros_aceptables']:,} ({report['resumen_ejecutivo']['indicadores_clave']['validacion_espacial']['porcentaje_aceptable']:.1f}%)",
    f"- **Registros Inválidos:** {report['resumen_ejecutivo']['indicadores_clave']['validacion_espacial']['registros_invalidos']:,}",
    f"- **Fuera de Límites:** {report['resumen_ejecutivo']['indicadores_clave']['validacion_espacial']['registros_fuera_limites']:,} registros",
    "",
    "---",
    "",
    "## 📈 Análisis Detallado",
    "",
    "### Procesamiento de Datos",
    f"- Registros cargados: {report['analisis_detallado']['procesamiento_datos']['registros_cargados']:,}",
    f"- Registros transformados: {report['analisis_detallado']['procesamiento_datos']['registros_transformados']:,}",
    f"- Columnas finales: {report['analisis_detallado']['procesamiento_datos']['columnas_finales']}",
    f"- UPID generados: {report['analisis_detallado']['procesamiento_datos']['upid_generados']:,}",
    "",
    "### Validación Presupuestal",
    f"- Con presupuesto: {report['analisis_detallado']['validacion_presupuestal']['registros_con_presupuesto']:,} registros",
    f"- Sin presupuesto: {report['analisis_detallado']['validacion_presupuestal']['registros_sin_presupuesto']:,} registros",
    f"- **Presupuesto Total:** {report['analisis_detallado']['validacion_presupuestal']['presupuesto_total']}",
    "",
    "### Procesamiento Geoespacial",
    f"- Geocodificados: {report['analisis_detallado']['procesamiento_geoespacial']['registros_geocodificados']:,}",
    f"- Sin geocodificar: {report['analisis_detallado']['procesamiento_geoespacial']['registros_sin_geocodificar']:,}",
    f"- Dentro de límites Cali: {report['analisis_detallado']['procesamiento_geoespacial']['registros_dentro_limites']:,}",
    f"- Fuera de límites: {report['analisis_detallado']['procesamiento_geoespacial']['registros_fuera_limites']:,}",
    f"- Sistema de coordenadas: `{report['analisis_detallado']['procesamiento_geoespacial']['sistema_coordenadas']}`",
    "",
    "### Normalización",
    f"- Valores de comuna normalizados: {report['analisis_detallado']['normalizacion']['valores_comuna_normalizados']:,}",
    f"- Valores de barrio normalizados: {report['analisis_detallado']['normalizacion']['valores_barrio_normalizados']:,}",
    f"- **Total normalizaciones:** {report['analisis_detallado']['normalizacion']['total_normalizaciones']:,}",
    "",
    "---",
    "",
    "## 🎯 Recomendaciones",
    ""
]

for i, rec in enumerate(report['recomendaciones'], 1):
    priority_emoji = "🔴" if rec['prioridad'] == "ALTA" else "🟡" if rec['prioridad'] == "MEDIA" else "🟢"
    md_lines.extend([
        f"### {i}. {rec['categoria']} {priority_emoji}",
        f"**Prioridad:** {rec['prioridad']}  ",
        f"**Problema:** {rec['issue']}  ",
        f"**Impacto:** {rec['impacto']}  ",
        f"**Recomendación:** {rec['recomendacion']}",
        ""
    ])

md_lines.extend([
    "---",
    "",
    "## ⚡ Acciones Prioritarias",
    ""
])

for action in report['acciones_prioritarias']:
    md_lines.extend([
        f"### Prioridad {action['prioridad']}",
        f"**Acción:** {action['accion']}  ",
        f"**Registros Afectados:** {action['registros_afectados']:,}  ",
        f"**Impacto Esperado:** {action['impacto_esperado']}",
        ""
    ])

md_lines.extend([
    "---",
    "",
    "## 📊 Métricas de Calidad",
    "",
    "### Completitud",
    f"- **Geométrica:** {report['metricas_calidad']['completitud']['geometrica']:.1f}%",
    f"- **Temporal:** {report['metricas_calidad']['completitud']['temporal']:.1f}%",
    f"- **Presupuestal:** {report['metricas_calidad']['completitud']['presupuestal']:.1f}%",
    "",
    "### Consistencia",
    f"- **Espacial:** {report['metricas_calidad']['consistencia']['espacial']:.1f}%",
    f"- **Temporal:** {report['metricas_calidad']['consistencia']['temporal']:.1f}%",
    f"- **Referencial:** {report['metricas_calidad']['consistencia']['referencial']:.1f}%",
    "",
    "### Precisión",
    f"- **Dentro de límites geográficos:** {report['metricas_calidad']['precision']['dentro_limites_geograficos']:.1f}%",
    f"- **Validación administrativa:** {report['metricas_calidad']['precision']['validacion_administrativa']:.1f}%",
    f"- **Normalización de nomenclatura:** {report['metricas_calidad']['precision']['normalizacion_nomenclatura']:.1f}%",
    "",
    "---",
    "",
    "## 📦 Exportación",
    f"- **Archivo:** `{Path(report['exportacion']['archivo_generado']).name if report['exportacion']['archivo_generado'] else 'N/A'}`",
    f"- **Formato:** {report['exportacion']['formato']}",
    f"- **Tamaño:** {report['exportacion']['tamano_kb']:.2f} KB",
    "",
    "---",
    "",
    f"*Reporte generado automáticamente - {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}*"
])

# Save Markdown report
report_md_file = report_output_dir / f'analisis_recomendaciones_{timestamp_str}.md'
with open(report_md_file, 'w', encoding='utf-8') as f:
    f.write('\n'.join(md_lines))

print(f"✓ Reporte Markdown guardado: {report_md_file.name}")
print(f"  Tamaño: {report_md_file.stat().st_size / 1024:.2f} KB")
print()

print("=" * 80)
print("✓ GENERACIÓN DE REPORTES COMPLETADA")
print("=" * 80)
print()
print(f"📂 Archivos generados:")
print(f"   - JSON: {report_json_file}")
print(f"   - Markdown: {report_md_file}")
print()
print(f"📊 Resumen:")
print(f"   - Calidad Global: {quality_score:.1f}% ({spatial_quality})")
print(f"   - Recomendaciones: {len(recommendations)}")
print(f"   - Acciones Prioritarias: {len(report['acciones_prioritarias'])}")

GENERANDO REPORTE DE ANÁLISIS Y RECOMENDACIONES

✓ Métricas cargadas desde: transformation_metrics_20251116_023227.json

✓ Reporte JSON guardado: analisis_recomendaciones_20251116_024112.json
  Tamaño: 7.36 KB

✓ Reporte Markdown guardado: analisis_recomendaciones_20251116_024112.md
  Tamaño: 5.35 KB

✓ GENERACIÓN DE REPORTES COMPLETADA

📂 Archivos generados:
   - JSON: a:\programing_workspace\proyectos_cali_alcaldia_etl\app_outputs\reports\analisis_recomendaciones_20251116_024112.json
   - Markdown: a:\programing_workspace\proyectos_cali_alcaldia_etl\app_outputs\reports\analisis_recomendaciones_20251116_024112.md

📊 Resumen:
   - Calidad Global: 67.7% (REGULAR)
   - Recomendaciones: 7
   - Acciones Prioritarias: 5
